**Проект**: Ed-Tech. Образовательный портал существующий уже 6 год, имеющий в своей продуктовой линейке вебинары, интенсивы, курсы, подписку, повышение квалификации, лид-магниты и трипвайеры.  

**Контекст**: Сформировалось понимание, что дальнейший рост невозможен без работы над метриками воронки. Масштабирование трафика уже не приводит к существенному росту прибыли.

**Задача**: Провести анализ пользовательского пути до покупки, исследовать пользовательский путь в зависимости от покупки и исследовать источники трафика

**Разделы исследования**: <a id='stepend'></a>

1. [Импорт необходимых библиотек](#step1)</n>

2. [Авторизация в google BigQuery](#step2)

3. [Запрос SQL на формирование таблицы с данными GA из Google BigQuery](#step3)

4. [Запрос SQL на формирование таблицы с данными покупателей](#step4)

5. [Запрос SQL на формирование таблицы с данными не покупателей](#step5)

6. [Query-запросы для формирования датафреймов](#step6)
  * [6.1 Query-запрос покупатели](#step6_1)
  * [6.2 Query-запрос не покупатели](#step5_2)

7. [Датафреймы для каждого типа постетителей](#step7)
  * [7.1 Query-запрос покупатели](#step7_1)
  * [7.2 Query-запрос не покупатели](#step7_2)

8. [Созданные датафреймы с необходимыми для анализа колонками](#step8)

9. [Исследование минимальной и максимальной даты первого визита](#step9)

10. [Исследование кол-ва дней между первым визитом и покупкой](#step10)

11. [Cреднее кол-во дней между регистрацией и покупкой по когортам](#step11)

12. [Cреднее кол-во визитов каждого типа продукта пользователей, кто не купил продукт, за все время жизни](#step12)

13. [Cравнения кол-ва визитов по типам страниц тех, кто купил и не купил](#step13)

14. [Cреднее кол-во визитов каждого типа продукта по когортам, среди купивших пользователей](#step14)

15. [Cреднее кол-во визитов каждого типа продукта по когортам, среди некупивших пользователей](#step15)

16. [Функция, показывающая среднее кол-во посещенных страниц по типу в зависимости от купленного продукта](#step16)

17. [Функция, показывающая среднее и медианное кол-во дней до покупки с разделением на когорты по месяцу первого визита](#step17)

18. [Примеры путей пользователей купивших вебинар и курс](#step18)
  * [18.1 Query-запрос покупатели](#step18_1)
  * [18.2 Query-запрос не покупатели](#step18_2)

19. [Пути пользователей до первой покупки с указанием продукта и кол-ва дней до покупки](#step19)
  * [19.1 Расcчет среднего периода до покупки среди покупателей июня по типам продуктов](#step19_1)
  * [19.2 Рассчет глобального среднего среди покупателей июня](#step19_2)

20. [Формирование данных для дальнейшего построения цепей Маркова](#step20)
  * [20.1 Query-запрос и датафрейм с данными пользователей, которые совершили покупку](#step20_1)
  * [20.2 Query-запрос и датафрейм с данными пользователей, которые не совершили покупку](#step20_2)

21. [Сформированные датафреймы на основе запросов п.20](#step21)
  * [21.1 Датафрейм с данными пользователей, которые совершили покупку](#step21_1)
  * [21.2 Датафрейм с данными пользователей, которые не совершили покупку](#step21_2)

22. [Цепи Маркова (длиннные) полный путь пользователей с объединением точек контакта после 10-ой](#step22)

23. [Цепи Маркова (короткие)](#step23)

24. [Выводы и дальнейшие шаги](#step24)

25. [Анализ поведения пользователей после первой покупки](https://nbviewer.org/github/artem-ilienkov/product_analysis_ed_tech/blob/main/ed_tech_sales.ipynb)

### `1. Импортируем необходимые библиотеки` <a id="step1"></a>



In [60]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime
from operator import attrgetter
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
pd.set_option('display.max_rows', 1000); pd.set_option('display.max_columns', 1000); pd.set_option('display.width', 1000)
!pip install markov-model-attribution


In [61]:
import markov_model_attribution as mma


### `2. Авторизуемся в Google BigQuery`<a id="step2"></a>

In [62]:
from google.colab import auth
auth.authenticate_user()
print('Авторизация пройдена')

Авторизация пройдена


In [63]:
from google.cloud import bigquery

project_id = 'test29102023'
client = bigquery.Client(project=project_id)


### `3. Сформируем таблицу с данными из Google BigQuery`<a id="step3"></a>


 [Вернуться в начало](#stepend)</n>

>**Комментарий**: *В дальнейшем для экономии объема запросов результат некоторых таблиц буду загружать в BQ и уже работать в дальнейшем с ней*


In [64]:
%%bigquery --project test29102023

# Таблица с пользователями c данными GA, где отсечены дубли страниц в рамках одного дня по каждому пользователю, чтобы исключить несколько сессий в один день
WITH visits AS (SELECT ga_id,
                       event_date,
                       event_name,
                       event_time,
                       page_title,
                       utm_campaign,
                       utm_medium,
                       utm_source,
                FROM (SELECT CAST (ga_id AS NUMERIC) AS ga_id,
                             CAST (event_date AS DATE) AS event_date,
                             event_name,
                             event_time,
                             page_title,
                             utm_campaign,
                             utm_medium,
                             utm_source,
                             # ранжируем страницы по дате посещения
                             ROW_NUMBER() OVER(PARTITION BY ga_id, page_title ORDER BY event_time ASC) AS rang
                      FROM (SELECT user_pseudo_id AS ga_id,
                                   # вытащим нужную инфу из подстрок
                                   CAST(CONCAT(SUBSTR(event_date, 0 , 4),'-' , SUBSTR(event_date, 5 , 2),'-' ,SUBSTR(event_date, 7 , 2)) AS DATE) AS event_date,
                                   # переведем секунды в дату события
                                   TIMESTAMP_SECONDS(CAST(CAST(event_timestamp as INT64)/1000000 AS INT64)) AS event_time,
                                   event_name,
                                   (SELECT value.string_value FROM UNNEST(event_params) WHERE key = 'page_title') AS page_title,
                                   (SELECT value.string_value FROM UNNEST(event_params) WHERE key = 'page_referrer') AS page_referrer,
                                   traffic_source.name AS utm_campaign,
                                   traffic_source.medium AS utm_medium,
                                   traffic_source.source AS utm_source
                             FROM `test29102023.logo.ga`
                            WHERE event_name = 'first_visit' OR event_name = 'page_view') AS tab_1
                      --WHERE ga_id = '1000486154.1679853384'
                      ORDER BY rang ASC
                      --LIMIT 30
                      ) AS tab_2
                      WHERE rang = 1)

SELECT *
FROM visits
LIMIT 10

Query is running:   0%|          |

Downloading:   0%|          |

,ga_id,event_date,event_name,event_time,page_title,utm_campaign,utm_medium,utm_source
0,1000344974.168694573,2023-06-16,page_view,2023-06-16 20:02:13+00:00,«Структура и содержание индивидуального логопедического занятия в ДОО»,16.06,Korrekcionniy,vk
1,100066516.167595210,2023-02-09,page_view,2023-02-09 14:15:05+00:00,"Центр дистанционного и онлайн обучения и переподготовки логопедов ""Лого-Эксперт""",(direct),(none),(direct)
2,1000737584.165293777,2023-04-11,page_view,2023-04-11 14:30:07+00:00,Страница входа,post_logomassage,4,vk
3,1000590352.167416428,2023-02-01,page_view,2023-02-01 10:14:35+00:00,"Диспраксия у детей: симптомы, виды и коррекция. Блог Лого-Эксперт",(referral),referral,yandex.ru
4,1000160057.168597215,2023-06-05,first_visit,2023-06-05 13:36:34+00:00,Монологическая речь: виды и особенности речи. Блог Лого-Эксперт,(referral),referral,yandex.ru
5,1000080417.168052565,2023-04-03,page_view,2023-04-03 12:41:16+00:00,Трансляция скоро начнется,(direct),(none),(direct)
6,1000175728.164996104,2023-05-22,page_view,2023-05-22 06:36:04+00:00,Страница входа,(direct),(none),(direct)
7,1000486154.167985338,2023-03-26,page_view,2023-03-26 18:01:54+00:00,Февраль,(direct),(none),(direct)
8,1000305884.168500750,2023-05-25,first_visit,2023-05-25 09:38:16+00:00,"Фонетико-фонематическое недоразвитие речи ФФНР у детей: причины, симптомы, диагностика и лечение болезни. Блог Лого-Эксперт",(referral),referral,ya.ru
9,1000175728.164996104,2023-04-26,page_view,2023-04-26 07:08:29+00:00,Документы,(direct),(none),(direct)


### `4. Сформируем таблицу с данными тех, кто купил`<a id="step4"></a>

 [Вернуться в начало](#stepend)</n>

In [65]:
%%bigquery --project test29102023

# 1. Таблица с пользователями с данными всех пользователей, результат таблицы visits
WITH users AS (SELECT ga_id,
                      event_date,
                      event_name,
                      event_time,
                      page_title,
                      RANK() OVER (PARTITION BY ga_id ORDER BY event_time ASC) AS rang,
                      utm_campaign,
                      utm_medium,
                      utm_source,
              FROM `test29102023.logo.visits_ga`
              WHERE ga_id IN (SELECT ga_id
                              FROM (SELECT ga_id,
                                          event_date,
                                          event_name,
                                          event_time,
                                          page_title,
                                          RANK() OVER (PARTITION BY ga_id ORDER BY event_time ASC) AS rang,
                                          utm_campaign,
                                          utm_medium,
                                          utm_source,
                                    FROM `test29102023.logo.visits_ga`
                                    WHERE (event_name = 'first_visit')
                                    AND (event_date >= '2022-11-04')) AS tab_2)),

# 2. Таблица с пользователями, у которых была хотя бы одна оплата
     buyers AS (SELECT *
                FROM `test29102023.logo.visits_ga`
                WHERE ga_id IN (SELECT ga_id
                                FROM (SELECT ga_id,
                                            SUM(b_status) AS b_status
                                      FROM (SELECT CAST(ga_id AS NUMERIC) AS ga_id,
                                                        event_date,
                                                        event_time,
                                                        page_title,
                                                        CAST((CASE
                                                          WHEN page_title LIKE '%Оплата%' THEN '1'
                                                          ELSE '0'
                                                        END) AS INT) AS b_status
                                                  FROM `test29102023.logo.visits_ga`
                                          ) AS tab_1
                                      GROUP BY ga_id
                                      HAVING b_status >= 1 ) AS tab_2)),

# 3. Таблица с пользователями, у которых не было покупок
      not_buyers AS (SELECT *
                  FROM `test29102023.logo.visits_ga`
                  WHERE ga_id IN (SELECT ga_id
                                  FROM (SELECT ga_id,
                                              SUM(b_status) AS b_status
                                        FROM (SELECT CAST(ga_id AS NUMERIC) AS ga_id,
                                                          event_date,
                                                          event_time,
                                                          page_title,
                                                          CAST((CASE
                                                            WHEN page_title LIKE '%Оплата%' THEN '1'
                                                            ELSE '0'
                                                          END) AS INT) AS b_status
                                                    FROM `test29102023.logo.visits_ga`
                                            ) AS tab_1
                                        GROUP BY ga_id
                                        HAVING b_status < 1 ) AS tab_2)),

# 4. Таблица, с указанием типа продукта по каждому названию
      type_products AS (SELECT string_field_0 AS page_title,
                             string_field_1 AS type
                       FROM `test29102023.logo.type_products`)


# 5. Таблица, в которой в отдельной колонке соединены тип продукта и его номер

SELECT ga_id,
       event_date,
       event_time,
       event_name,
       page_title,
       rang,
       type,
       rn,
       CONCAT(type, '', rn) AS product_way,
FROM (# 5.1 Таблица, в которой проранжированы посещения страниц в рамках каждого типа продукта
        SELECT *,
              ROW_NUMBER() over (partition by ga_id, type order by event_time) as rn,
        FROM (# 5.2 Таблица, в которой убраны все строки, которые идут после первой оплаты
              #+ добавлена колонка типа продукта
              SELECT tab_1.ga_id AS ga_id, # Уникальный идентификатор пользователя в GA
                    tab_1.event_date AS event_date, # Дата визита
                    tab_1.event_name AS event_name, # Характеристика визита
                    tab_1.event_time AS event_time, # Дата и время визита
                    tab_1.page_title AS page_title, # Название страницы визита
                    tab_1.rang AS rang, # Номер страницы посещения в рамках пользователя, нумерация каждого типа продукта, ранжирование по event_time ASC
                    tab_1.utm_campaign AS utm_campaign, # метка utm_campaign
                    tab_1.utm_medium AS utm_medium, # метка utm_medium
                    tab_1.utm_source AS utm_source, # метка utm_source
                    tab_3.type AS type # тип продукта
              FROM users AS tab_1
              INNER JOIN (# 5.3 Таблица, в sm начинается нумерация с с первой оплаты по каждому пользователю и исключает пользователей к которых
                          SELECT ga_id,
                                event_date,
                                event_name,
                                event_time,
                                page_title,
                                rang,
                                sm
                          FROM (SELECT ga_id,
                                      event_date,
                                      event_name,
                                      event_time,
                                      page_title,
                                      RANK() OVER (PARTITION BY ga_id ORDER BY event_time ASC) AS rang,
                                      SUM(CASE WHEN page_title LIKE '%Оплата%' THEN 1 ELSE 0 END) OVER (PARTITION BY ga_id ORDER BY event_time) AS sm # суммируем покупки по окну со строкой "Оплата"
                                FROM `test29102023.logo.visits_ga`) AS tab_2
                          --WHERE ga_id = 10603501.167690103
                          )
              AS tab_2 ON tab_2.ga_id = tab_1.ga_id AND tab_1.rang <= tab_2.rang

              LEFT JOIN type_products AS tab_3 ON tab_3.page_title = tab_1.page_title
              # В следующей строке условие tab_2.sm = 1 исключает пользователей без оплаты (если поставить =2, то будет путь пользователя до второй оплаты)
              WHERE tab_2.sm = 1 AND tab_2.page_title LIKE '%Оплата%') AS tab_4
        ORDER BY rang ASC) AS tab_5
  WHERE (page_title != 'None')
  AND (tab_5.type != 'Техническое')
  AND (tab_5.type != 'Тестирование')
  AND (tab_5.type != 'Удостоверение')
  AND (tab_5.type != 'Заявка')
  AND (tab_5.type != 'Заказ')
  AND (page_title != 'None')
  AND (tab_5.type != 'Покупка')
  AND (tab_5.type != 'Лекция')


Query is running:   0%|          |

Downloading:   0%|          |

,ga_id,event_date,event_time,event_name,page_title,rang,type,rn,product_way
0,76400733.163645229,2022-11-04,2022-11-04 12:55:18+00:00,first_visit,Методические рекомендации к логопедическому массажу,1,Мастер-класс,1,Мастер-класс1
1,139582063.164999884,2022-11-26,2022-11-26 20:25:47+00:00,first_visit,«Структура и содержание индивидуального логопедического занятия в ДОО»,1,Интенсив,1,Интенсив1
2,147557850.167947315,2023-03-22,2023-03-22 08:19:11+00:00,first_visit,Запись,1,Вебинар_апселл,1,Вебинар_апселл1
3,169898743.168380969,2023-05-11,2023-05-11 12:54:53+00:00,first_visit,Работа логопеда в школе — эффективно и системно,1,Подписка,1,Подписка1
4,189417422.164774123,2022-11-08,2022-11-08 17:15:57+00:00,first_visit,Чемоданчик логопеда | Дошкольная Академия,1,Мастер-класс,1,Мастер-класс1
...,...,...,...,...,...,...,...,...,...
44206,22276846.166194398,2023-04-19,2023-04-19 08:22:04+00:00,page_view,Документы для лекции по алалии. 1 страница,295,Лид-магнит,14,Лид-магнит14
44207,22276846.166194398,2023-04-20,2023-04-20 07:28:18+00:00,page_view,Документы для лекции по алалии. 2 страница,296,Лид-магнит,15,Лид-магнит15
44208,22276846.166194398,2023-05-14,2023-05-14 11:51:15+00:00,page_view,Использование нейромеханизмов в коррекционной работе,297,Мастер-класс,57,Мастер-класс57
44209,22276846.166194398,2023-05-15,2023-05-15 11:26:43+00:00,page_view,"8 способов преподнесения артикуляционной гимнастики. Как сделать так, чтобы ребенку не было скучно?",298,Трипвайер,36,Трипвайер36


### `5. Сформируем таблицу с данными тех, кто не купил`<a id="step5"></a>

 [Вернуться в начало](#stepend)</n>

In [66]:
%%bigquery --project test29102023

# 1. Таблица с пользователями с данными всех пользователей, результат таблицы visits
WITH users AS (SELECT ga_id,
                      event_date,
                      event_name,
                      event_time,
                      page_title,
                      RANK() OVER (PARTITION BY ga_id ORDER BY event_time ASC) AS rang,
                      utm_campaign,
                      utm_medium,
                      utm_source,
              FROM `test29102023.logo.visits_ga`
              WHERE ga_id IN (SELECT ga_id
                              FROM (SELECT ga_id,
                                          event_date,
                                          event_name,
                                          event_time,
                                          page_title,
                                          RANK() OVER (PARTITION BY ga_id ORDER BY event_time ASC) AS rang,
                                          utm_campaign,
                                          utm_medium,
                                          utm_source,
                                    FROM `test29102023.logo.visits_ga`
                                    WHERE (event_name = 'first_visit')))),
                                    --AND (event_date >= '2023-06-01')) AS tab_2)

# 2. Таблица с пользователями, у которых была хотя бы одна оплата
     buyers AS (SELECT *
                FROM `test29102023.logo.visits_ga`
                WHERE ga_id IN (SELECT ga_id
                                FROM (SELECT ga_id,
                                            SUM(b_status) AS b_status
                                      FROM (SELECT CAST(ga_id AS NUMERIC) AS ga_id,
                                                        event_date,
                                                        event_time,
                                                        page_title,
                                                        CAST((CASE
                                                          WHEN page_title LIKE '%Оплата%' THEN '1'
                                                          ELSE '0'
                                                        END) AS INT) AS b_status
                                                  FROM `test29102023.logo.visits_ga`
                                          ) AS tab_1
                                      GROUP BY ga_id
                                      HAVING b_status >= 1 ) AS tab_2)),

# 3. Таблица с пользователями, у которых не было покупок
      not_buyers AS (SELECT ga_id
                     FROM `test29102023.logo.visits_ga`
                     WHERE ga_id IN (SELECT ga_id
                                     FROM (SELECT ga_id,
                                                  SUM(b_status) AS b_status
                                           FROM (SELECT CAST(ga_id AS NUMERIC) AS ga_id,
                                                        event_date,
                                                        event_time,
                                                        page_title,
                                                        CAST((CASE
                                                          WHEN page_title LIKE '%Оплата%' THEN '1'
                                                          ELSE '0'
                                                        END) AS INT) AS b_status
                                                  FROM `test29102023.logo.visits_ga`
                                          ) AS tab_1
                                      GROUP BY ga_id
                                      HAVING b_status < 1 ) AS tab_2)),

# 4. Таблица, с указанием типа продукта по каждому названию
      type_products AS (SELECT string_field_0 AS page_title,
                               string_field_1 AS type
                       FROM `test29102023.logo.type_products`)

# 5. Таблица, в которой в отдельной колонке соединены тип продукта и его номер
SELECT *,
       CONCAT(type, '', rn) AS product_way
FROM (
SELECT tab_1.ga_id AS ga_id,
       tab_1.event_date AS event_date,
       tab_1.event_name AS event_name,
       tab_1.event_time AS event_time,
       tab_1.page_title AS page_title,
       tab_1.rang AS rang ,
       tab_1.utm_campaign AS utm_campaign ,
       tab_1.utm_medium AS utm_medium ,
       tab_1.utm_source AS utm_source ,
       tab_3.type AS type ,
       ROW_NUMBER() over (partition by tab_1.ga_id, type order by tab_1.event_time) as rn
FROM (# 5.1 Таблица, в которой проранжированы посещения страниц в рамках каждого типа продукта
      SELECT *
      FROM users
      WHERE ga_id IN (SELECT ga_id
                     FROM `test29102023.logo.visits_ga`
                     WHERE ga_id IN (SELECT ga_id
                                     FROM (SELECT ga_id,
                                                  SUM(b_status) AS b_status
                                           FROM (SELECT CAST(ga_id AS NUMERIC) AS ga_id,
                                                        event_date,
                                                        event_time,
                                                        page_title,
                                                        CAST((CASE
                                                          WHEN page_title LIKE '%Оплата%' THEN '1'
                                                          ELSE '0'
                                                        END) AS INT) AS b_status
                                                  FROM `test29102023.logo.visits_ga`
                                          ) AS tab_1
                                      GROUP BY ga_id
                                      HAVING b_status < 1 ) AS tab_2))) AS tab_1
LEFT JOIN type_products AS tab_3 ON tab_3.page_title = tab_1.page_title
WHERE (tab_1.page_title != 'None')
AND (tab_3.type != 'Техническое')
AND (tab_3.type != 'Тестирование')
AND (tab_3.type != 'Удостоверение')
AND (tab_3.type != 'Заявки на вебинар')
AND (tab_3.type != 'Заявка')
AND (tab_3.type != 'Заказ')
AND (tab_1.page_title != 'None')
AND (tab_3.type != 'Покупка')
AND (tab_3.type != 'Лекция')
) AS main_tab


Query is running:   0%|          |

Downloading:   0%|          |

,ga_id,event_date,event_name,event_time,page_title,rang,utm_campaign,utm_medium,utm_source,type,rn,product_way
0,4499073.167872074,2023-03-16,page_view,2023-03-15 21:27:40+00:00,Планирование индивидуальной работы логопеда ДОУ с ребенком с ЗРР,10,(referral),referral,click.mail.ru,Мастер-класс,4,Мастер-класс4
1,8659691.167429967,2023-02-24,page_view,2023-02-24 02:43:32+00:00,"Онлайн вебинары для логопедов, дефектологов, воспитателей, учителей и психологов",29,maraphon_18012023,email,newsletter,Вебинар,2,Вебинар2
2,8659691.167429967,2023-03-01,page_view,2023-03-01 10:47:07+00:00,"Общая схема коррекции нарушений звукопроизношений. Дефекты оглушения, озвончения, смягчения и их исправление",55,maraphon_18012023,email,newsletter,Вебинар,3,Вебинар3
3,20584288.166810368,2022-11-10,page_view,2022-11-10 18:08:51+00:00,Практические материалы для работы логопедов и коррекционных специалистов,3,(referral),referral,away.vk.com,Курс,2,Курс2
4,30004503.167489648,2023-03-24,page_view,2023-03-24 05:40:34+00:00,Логотека,15,(referral),referral,yandex.ru,Курс,2,Курс2
...,...,...,...,...,...,...,...,...,...,...,...,...
667505,1940273851.168619529,2023-06-08,page_view,2023-06-08 03:35:47+00:00,Запись,2,(referral),referral,m.vk.com,Вебинар_апселл,1,Вебинар_апселл1
667506,2007838632.168245269,2023-04-25,first_visit,2023-04-25 19:58:07+00:00,Запись,1,(direct),(none),(direct),Вебинар_апселл,1,Вебинар_апселл1
667507,2054979312.167800318,2023-03-07,page_view,2023-03-07 09:32:38+00:00,Запись,3,(direct),(none),(direct),Вебинар_апселл,1,Вебинар_апселл1
667508,2060433521.161020339,2023-04-25,page_view,2023-04-25 15:34:45+00:00,Запись,2,(direct),(none),(direct),Вебинар_апселл,1,Вебинар_апселл1


### `6. Сформируем запросы для формирования датафреймов в Pandas` <a id="step6"></a>

 [Вернуться в начало](#stepend)</n>

6.1 Запрос с формированием покупателей <a id="step6_1"></a>

 [Вернуться в начало](#stepend)</n>

In [67]:
QUERY_1 = ('''
# 1. Таблица с пользователями с данными всех пользователей, результат таблицы visits
WITH users AS (SELECT ga_id,
                      event_date,
                      event_name,
                      event_time,
                      page_title,
                      RANK() OVER (PARTITION BY ga_id ORDER BY event_time ASC) AS rang,
                      utm_campaign,
                      utm_medium,
                      utm_source,
              FROM `test29102023.logo.visits_ga`
              WHERE ga_id IN (SELECT ga_id
                              FROM (SELECT ga_id,
                                          event_date,
                                          event_name,
                                          event_time,
                                          page_title,
                                          RANK() OVER (PARTITION BY ga_id ORDER BY event_time ASC) AS rang,
                                          utm_campaign,
                                          utm_medium,
                                          utm_source,
                                    FROM `test29102023.logo.visits_ga`
                                    WHERE (event_name = 'first_visit')
                                    AND (event_date >= '2022-11-04')) AS tab_2)),

# 2. Таблица с пользователями, у которых была хотя бы одна оплата
     buyers AS (SELECT *
                FROM `test29102023.logo.visits_ga`
                WHERE ga_id IN (SELECT ga_id
                                FROM (SELECT ga_id,
                                            SUM(b_status) AS b_status
                                      FROM (SELECT CAST(ga_id AS NUMERIC) AS ga_id,
                                                        event_date,
                                                        event_time,
                                                        page_title,
                                                        CAST((CASE
                                                          WHEN page_title LIKE '%Оплата%' THEN '1'
                                                          ELSE '0'
                                                        END) AS INT) AS b_status
                                                  FROM `test29102023.logo.visits_ga`
                                          ) AS tab_1
                                      GROUP BY ga_id
                                      HAVING b_status >= 1 ) AS tab_2)),

# 3. Таблица с пользователями, у которых не было покупок
      not_buyers AS (SELECT *
                  FROM `test29102023.logo.visits_ga`
                  WHERE ga_id IN (SELECT ga_id
                                  FROM (SELECT ga_id,
                                              SUM(b_status) AS b_status
                                        FROM (SELECT CAST(ga_id AS NUMERIC) AS ga_id,
                                                          event_date,
                                                          event_time,
                                                          page_title,
                                                          CAST((CASE
                                                            WHEN page_title LIKE '%Оплата%' THEN '1'
                                                            ELSE '0'
                                                          END) AS INT) AS b_status
                                                    FROM `test29102023.logo.visits_ga`
                                            ) AS tab_1
                                        GROUP BY ga_id
                                        HAVING b_status < 1 ) AS tab_2)),

# 4. Таблица, с указанием типа продукта по каждому названию
      type_products AS (SELECT string_field_0 AS page_title,
                             string_field_1 AS type
                       FROM `test29102023.logo.type_products`)


# 5. Таблица, в которой в отдельной колонке соединены тип продукта и его номер

SELECT ga_id,
       event_date,
       event_time,
       event_name,
       page_title,
       rang,
       type,
       rn,
       CONCAT(type, '', rn) AS product_way,
FROM (# 5.1 Таблица, в которой проранжированы посещения страниц в рамках каждого типа продукта
        SELECT *,
              ROW_NUMBER() over (partition by ga_id, type order by event_time) as rn,
        FROM (# 5.2 Таблица, в которой убраны все строки, которые идут после первой оплаты + добавлена колонка типа продукта
              SELECT tab_1.ga_id AS ga_id, # Уникальный идентификатор пользователя в GA
                    tab_1.event_date AS event_date, # Дата визита
                    tab_1.event_name AS event_name, # Характеристика визита
                    tab_1.event_time AS event_time, # Дата и время визита
                    tab_1.page_title AS page_title, # Название страницы визита
                    tab_1.rang AS rang, # Номер страницы посещения в рамках пользователя, нумерация каждого типа продукта, ранжирование по event_time ASC
                    tab_1.utm_campaign AS utm_campaign, # метка utm_campaign
                    tab_1.utm_medium AS utm_medium, # метка utm_medium
                    tab_1.utm_source AS utm_source, # метка utm_source
                    tab_3.type AS type # тип продукта
              FROM users AS tab_1
              INNER JOIN (# 5.3 Таблица, в sm начинается нумерация с с первой оплаты по каждому пользователю и исключает пользователей к которых
                          SELECT ga_id,
                                event_date,
                                event_name,
                                event_time,
                                page_title,
                                rang,
                                sm
                          FROM (SELECT ga_id,
                                      event_date,
                                      event_name,
                                      event_time,
                                      page_title,
                                      RANK() OVER (PARTITION BY ga_id ORDER BY event_time ASC) AS rang,
                                      SUM(CASE WHEN page_title LIKE '%Оплата%' THEN 1 ELSE 0 END) OVER (PARTITION BY ga_id ORDER BY event_time) AS sm # суммируем покупки по окну со строкой "Оплата"
                                FROM `test29102023.logo.visits_ga`) AS tab_2
                          --WHERE ga_id = 10603501.167690103
                          )
              AS tab_2 ON tab_2.ga_id = tab_1.ga_id AND tab_1.rang <= tab_2.rang

              LEFT JOIN type_products AS tab_3 ON tab_3.page_title = tab_1.page_title
              # В следующей строке условие tab_2.sm = 1 исключает пользователей без оплаты (если поставить =2, то будет путь пользователя до второй оплаты)
              WHERE tab_2.sm = 1 AND tab_2.page_title LIKE '%Оплата%') AS tab_4
        ORDER BY rang ASC) AS tab_5
  WHERE (page_title != 'None')
  AND (tab_5.type != 'Техническое')
  AND (tab_5.type != 'Тестирование')
  AND (tab_5.type != 'Удостоверение')
  AND (tab_5.type != 'Заявка')
  AND (tab_5.type != 'Заказ')
  AND (page_title != 'None')
  AND (tab_5.type != 'Покупка')
  AND (tab_5.type != 'Лекция')
''')

6.2 Запрос с формированием не покупателей <a id="step6_2"></a>


 [Вернуться в начало](#stepend)</n>

In [68]:
QUERY_2 = ('''
# 1. Таблица с пользователями с данными всех пользователей, результат таблицы visits
WITH users AS (SELECT ga_id,
                      event_date,
                      event_name,
                      event_time,
                      page_title,
                      RANK() OVER (PARTITION BY ga_id ORDER BY event_time ASC) AS rang,
                      utm_campaign,
                      utm_medium,
                      utm_source,
              FROM `test29102023.logo.visits_ga`
              WHERE ga_id IN (SELECT ga_id
                              FROM (SELECT ga_id,
                                          event_date,
                                          event_name,
                                          event_time,
                                          page_title,
                                          RANK() OVER (PARTITION BY ga_id ORDER BY event_time ASC) AS rang,
                                          utm_campaign,
                                          utm_medium,
                                          utm_source,
                                    FROM `test29102023.logo.visits_ga`
                                    WHERE (event_name = 'first_visit')))),
                                    --AND (event_date >= '2023-06-01')) AS tab_2)

# 2. Таблица с пользователями, у которых была хотя бы одна оплата
     buyers AS (SELECT *
                FROM `test29102023.logo.visits_ga`
                WHERE ga_id IN (SELECT ga_id
                                FROM (SELECT ga_id,
                                            SUM(b_status) AS b_status
                                      FROM (SELECT CAST(ga_id AS NUMERIC) AS ga_id,
                                                        event_date,
                                                        event_time,
                                                        page_title,
                                                        CAST((CASE
                                                          WHEN page_title LIKE '%Оплата%' THEN '1'
                                                          ELSE '0'
                                                        END) AS INT) AS b_status
                                                  FROM `test29102023.logo.visits_ga`
                                          ) AS tab_1
                                      GROUP BY ga_id
                                      HAVING b_status >= 1 ) AS tab_2)),

# 3. Таблица с пользователями, у которых не было покупок
      not_buyers AS (SELECT ga_id
                     FROM `test29102023.logo.visits_ga`
                     WHERE ga_id IN (SELECT ga_id
                                     FROM (SELECT ga_id,
                                                  SUM(b_status) AS b_status
                                           FROM (SELECT CAST(ga_id AS NUMERIC) AS ga_id,
                                                        event_date,
                                                        event_time,
                                                        page_title,
                                                        CAST((CASE
                                                          WHEN page_title LIKE '%Оплата%' THEN '1'
                                                          ELSE '0'
                                                        END) AS INT) AS b_status
                                                  FROM `test29102023.logo.visits_ga`
                                          ) AS tab_1
                                      GROUP BY ga_id
                                      HAVING b_status < 1 ) AS tab_2)),

# 4. Таблица, с указанием типа продукта по каждому названию
      type_products AS (SELECT string_field_0 AS page_title,
                               string_field_1 AS type
                       FROM `test29102023.logo.type_products`)

# 5. Таблица, в которой в отдельной колонке соединены тип продукта и его номер
SELECT *,
       CONCAT(type, '', rn) AS product_way
FROM (
SELECT tab_1.ga_id AS ga_id,
       tab_1.event_date AS event_date,
       tab_1.event_name AS event_name,
       tab_1.event_time AS event_time,
       tab_1.page_title AS page_title,
       tab_1.rang AS rang ,
       tab_1.utm_campaign AS utm_campaign ,
       tab_1.utm_medium AS utm_medium ,
       tab_1.utm_source AS utm_source ,
       tab_3.type AS type ,
       ROW_NUMBER() over (partition by tab_1.ga_id, type order by tab_1.event_time) as rn
FROM (# 5.1 Таблица, в которой проранжированы посещения страниц в рамках каждого типа продукта
      SELECT *
      FROM users
      WHERE ga_id IN (SELECT ga_id
                     FROM `test29102023.logo.visits_ga`
                     WHERE ga_id IN (SELECT ga_id
                                     FROM (SELECT ga_id,
                                                  SUM(b_status) AS b_status
                                           FROM (SELECT CAST(ga_id AS NUMERIC) AS ga_id,
                                                        event_date,
                                                        event_time,
                                                        page_title,
                                                        CAST((CASE
                                                          WHEN page_title LIKE '%Оплата%' THEN '1'
                                                          ELSE '0'
                                                        END) AS INT) AS b_status
                                                  FROM `test29102023.logo.visits_ga`
                                          ) AS tab_1
                                      GROUP BY ga_id
                                      HAVING b_status < 1 ) AS tab_2))) AS tab_1
LEFT JOIN type_products AS tab_3 ON tab_3.page_title = tab_1.page_title
WHERE (tab_1.page_title != 'None')
AND (tab_3.type != 'Техническое')
AND (tab_3.type != 'Тестирование')
AND (tab_3.type != 'Удостоверение')
AND (tab_3.type != 'Заявки на вебинар')
AND (tab_3.type != 'Заявка')
AND (tab_3.type != 'Заказ')
AND (tab_1.page_title != 'None')
AND (tab_3.type != 'Покупка')
AND (tab_3.type != 'Лекция')
) AS main_tab
''')

### `7. Создадим датафреймы для каждого типа постетителей - покупатели и не покупатели`<a id="step7"></a>

 [Вернуться в начало](#stepend)</n>

7.1 Создадим датафрейм с данными покупателей <a id="step7_1"></a>

 [Вернуться в начало](#stepend)</n>

In [69]:
buyers = client.query(QUERY_1).to_dataframe()
# buyers.to_csv('buyers.csv', index = False)


7.2 Создадим датафрейм с данными не покупателей <a id="step7_2"></a>

 [Вернуться в начало](#stepend)</n>

In [70]:
no_buyers = client.query(QUERY_2).to_dataframe()
# no_buyers.to_csv('no_buyers.csv', index = False)

### `8. Оставим в датафреймах только нужные колонки` <a id="step8"></a>

 [Вернуться в начало](#stepend)</n>

In [71]:
buyers = buyers[['ga_id','event_date','event_time', 'page_title','rang','type','rn', 'product_way']]
no_buyers = no_buyers[['ga_id','event_date','event_time', 'page_title', 'rang','type','rn', 'product_way']]

### `9. Исследуем минимальную и максимальную дату первого визита`<a id="step9"></a>

 [Вернуться в начало](#stepend)</n>

>**Комментарий**: *Это даст понимание верхней и нижней границы дат в датафреймах*


In [72]:
buyers['ga_id'] = buyers['ga_id'].astype(str)
buyers['event_date'] = buyers['event_date'].apply(pd.to_datetime )
buyers['event_time'] = pd.to_datetime(buyers.event_time).dt.tz_localize(None)

no_buyers['ga_id'] = no_buyers['ga_id'].astype(str)
no_buyers['event_date'] = no_buyers['event_date'].apply(pd.to_datetime )
no_buyers['event_time'] = pd.to_datetime(no_buyers.event_time).dt.tz_localize(None)


<ipython-input-72-dbae0c0d0290>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buyers['ga_id'] = buyers['ga_id'].astype(str)


In [73]:
print('Минимальная дата первого визита(покупатели):', buyers.event_date.min())
print('Максимальная дата первого визита(покупатели):', buyers.event_date.max())
print()
print('Минимальная дата первого визита(не покупатели):', no_buyers.event_date.min())
print('Максимальная дата первого визита(не покупатели):', no_buyers.event_date.max())

Минимальная дата первого визита(покупатели): 2022-11-04 00:00:00
Максимальная дата первого визита(покупатели): 2023-06-21 00:00:00

Минимальная дата первого визита(не покупатели): 2022-11-04 00:00:00
Максимальная дата первого визита(не покупатели): 2023-06-21 00:00:00


In [74]:
buyers

,ga_id,event_date,event_time,page_title,rang,type,rn,product_way
0,4562680.162978025,2023-01-11,2023-01-11 03:07:33,"Центр дистанционного и онлайн обучения и переподготовки логопедов ""Лого-Эксперт""",1,Главная,1,Главная1
1,35729420.167335499,2023-01-10,2023-01-10 12:49:51,Логопедическая работа при устранении дефекта озвончения и смягчения,1,Подписка,1,Подписка1
2,41326112.167316121,2023-01-08,2023-01-08 07:00:08,(Не)простая коррекция звукопроизношения — секреты успеха логопеда,1,Трипвайер,1,Трипвайер1
3,69469118.168205393,2023-04-21,2023-04-21 05:12:13,"Центр дистанционного и онлайн обучения и переподготовки логопедов ""Лого-Эксперт""",1,Главная,1,Главная1
4,95888399.168240656,2023-04-25,2023-04-25 07:09:20,Оплата заказа #8282613,1,Оплата,1,Оплата1
...,...,...,...,...,...,...,...,...
44206,22276846.166194398,2023-04-19,2023-04-19 08:22:04,Документы для лекции по алалии. 1 страница,295,Лид-магнит,14,Лид-магнит14
44207,22276846.166194398,2023-04-20,2023-04-20 07:28:18,Документы для лекции по алалии. 2 страница,296,Лид-магнит,15,Лид-магнит15
44208,22276846.166194398,2023-05-14,2023-05-14 11:51:15,Использование нейромеханизмов в коррекционной работе,297,Мастер-класс,57,Мастер-класс57
44209,22276846.166194398,2023-05-15,2023-05-15 11:26:43,"8 способов преподнесения артикуляционной гимнастики. Как сделать так, чтобы ребенку не было скучно?",298,Трипвайер,36,Трипвайер36


### `10. Исследуем кол-во дней между первым визитом и покупкой`<a id="step10"></a>

 [Вернуться в начало](#stepend)</n>

In [75]:
client_first_date = buyers.groupby('ga_id')['event_date'].min().reset_index()
client_first_date = client_first_date.rename(columns={'event_date': 'first_date'})
buyers = buyers.merge(client_first_date, how= 'left', on ='ga_id')


client_last_date = buyers.groupby('ga_id')['event_date'].max().reset_index()
client_last_date = client_last_date.rename(columns={'event_date': 'last_date'})
buyers = buyers.merge(client_last_date, how= 'left', on ='ga_id')

buyers['cohort'] = buyers['first_date'].dt.to_period('M')

buyers['day_btw'] = (buyers['last_date'] - buyers['first_date']).dt.days

In [76]:
no_client_first_date = no_buyers.groupby('ga_id')['event_date'].min().reset_index()
no_client_first_date = no_client_first_date.rename(columns={'event_date': 'first_date'})
no_client_last_date = no_buyers.groupby('ga_id')['event_date'].max().reset_index()
no_client_last_date = no_client_last_date.rename(columns={'event_date': 'last_date'})

no_buyers = no_buyers.merge(no_client_first_date, how= 'left', on ='ga_id')
no_buyers = no_buyers.merge(no_client_last_date, how= 'left', on ='ga_id')

no_buyers['cohort'] = no_buyers['first_date'].dt.to_period('M')

no_buyers['day_btw'] = (no_buyers['last_date'] - no_buyers['first_date']).dt.days

In [77]:
print('Минимальное кол-во дней между регистрацией и первой продажей (покупатели):', buyers.day_btw.min())
print('Среднее кол-во дней между регистрацией и первой продажей (покупатели):', buyers.day_btw.mean().round())
print('Максимальное кол-во дней между регистрацией и первой продажей (покупатели):', buyers.day_btw.max())

Минимальное кол-во дней между регистрацией и первой продажей (покупатели): 0
Среднее кол-во дней между регистрацией и первой продажей (покупатели): 52.0
Максимальное кол-во дней между регистрацией и первой продажей (покупатели): 227


### `11. Найдем среднее кол-во дней между регистрацией и покупкой по когортам`<a id="step11"></a>

 [Вернуться в начало](#stepend)</n>

In [78]:
avg_days = buyers.copy()
avg_days = avg_days.groupby('cohort') \
                   .agg({'day_btw':['mean','median','max']}) \
                   .round(2) \
                   .reset_index()
avg_days

cohort day_btw            
              mean median  max
0  2022-11  74.06   47.0   227
1  2022-12  59.84   35.0   194
2  2023-01  41.87   18.0   168
3  2023-02  38.88   21.0   138
4  2023-03  25.79   9.0    104
5  2023-04  22.23   17.0   73 
6  2023-05  7.51    1.0    49 
7  2023-06  1.84    0.0    19

>**Комментарий**: *От когорте к когорте снижается время до первой продажи. Наиболее качественное изменение произошло в когорте 2023-05*

### `12. Найдем среднее кол-во визитов страниц по типам продукта до покупки`<a id="step12"></a>

 [Вернуться в начало](#stepend)</n>

In [79]:
avg_buyers =  buyers.copy()
avg_buyers = avg_buyers.groupby(['type']) \
                       .agg({'rn':'mean'}) \
                       .rename(columns={'rn': 'avg_visits_buyers'}) \
                       .round() \
                       .sort_values('avg_visits_buyers', ascending=False) \
                       .reset_index()
avg_buyers

,type,avg_visits_buyers
0,Нескучная логопедия,8.0
1,Блог,6.0
2,Вебинар,4.0
3,Мастер-класс,3.0
4,Курс,2.0
5,Интенсив,2.0
6,Подписка,2.0
7,Лид-магнит,2.0
8,Трипвайер,2.0
9,Заявки на вебинар,2.0


>**Комментарий**: *Вероятнее всего ключевую роль в принятии решения о покупке играет Блог, который является связующим звеном между страницами покупок. Нужно проработать эту страницу с целью перевода пользователя на страницы следующего уровня*


### `12.1 Рассчитаем среднее кол-во визитов каждого типа продукта пользователей, кто не купил продукт, за все время жизни`

In [80]:
avg_no_buyers =  no_buyers.copy()
avg_no_buyers = avg_no_buyers.groupby(['type']) \
                             .agg({'rn':'mean'}) \
                             .rename(columns={'rn': 'avg_visits_no_buyers'}) \
                             .round() \
                             .sort_values('avg_visits_no_buyers', ascending=False) \
                             .reset_index()
avg_no_buyers

,type,avg_visits_no_buyers
0,Нескучная логопедия,8.0
1,Блог,3.0
2,Вебинар,3.0
3,Курс,2.0
4,Подписка,2.0
5,Мастер-класс,2.0
6,Лид-магнит,2.0
7,Трипвайер,2.0
8,Интенсив,2.0
9,Конференция,1.0


### `13. Сформируем таблицу для сравнения кол-ва визитов по типам страниц, тех, кто купил и не купил`<a id="step13"></a>

 [Вернуться в начало](#stepend)</n>

In [81]:
avg_buyers = avg_buyers.merge(avg_no_buyers, how='left', on='type')
avg_buyers

,type,avg_visits_buyers,avg_visits_no_buyers
0,Нескучная логопедия,8.0,8.0
1,Блог,6.0,3.0
2,Вебинар,4.0,3.0
3,Мастер-класс,3.0,2.0
4,Курс,2.0,2.0
5,Интенсив,2.0,2.0
6,Подписка,2.0,2.0
7,Лид-магнит,2.0,2.0
8,Трипвайер,2.0,2.0
9,Заявки на вебинар,2.0,<NA>


>**Комментарий**: *Сравнивая посещения страниц по типам продукта, ключевая разница между теми, кто купил и теми, кто не купил в кол-ве посещений страниц блога. 6 страниц у купивших и 3 у не купивших*


>**Задача**: *Нужно провести качественные исследования купивших и не купивших, какую работу выполняет каждая страница, чего не хватает/хватает. Изучить поведение на странице блога и сгенерировать гипотезы переработки страницы блога, — добавив элементы перевода на смежные страницы, продукты, с разделением по направлениям и экспертам*

### `14. Рассчитаем среднее кол-во визитов каждого типа продукта по когортам, среди купивших пользователей`<a id="step14"></a>

 [Вернуться в начало](#stepend)</n>

In [82]:
avg_types = buyers.copy()
avg_types = avg_types.groupby(['cohort', 'type']) \
                     .agg({'rn':'mean'}).round().reset_index()
avg_types = avg_types.pivot_table(index='type',
                                  columns='cohort', values='rn')
avg_types

cohort,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06
type,,,,,,,,
Акция,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
Блог,6.0,6.0,4.0,3.0,3.0,15.0,11.0,1.0
Бонусы,1.0,1.0,1.0,1.0,1.0,1.0,1.0,<NA>
Вебинар,3.0,13.0,2.0,3.0,2.0,2.0,2.0,2.0
Вебинар_апселл,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
Вебинары,<NA>,<NA>,<NA>,1.0,<NA>,<NA>,<NA>,<NA>
Главная,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
Заявки на вебинар,2.0,<NA>,<NA>,<NA>,1.0,<NA>,<NA>,<NA>
Интенсив,2.0,2.0,2.0,3.0,2.0,2.0,1.0,1.0


### `15. Рассчитаем среднее кол-во визитов каждого типа продукта по когортам, среди некупивших пользователей`<a id="step15"></a>

 [Вернуться в начало](#stepend)</n>

In [83]:
avg_types = no_buyers.copy()
avg_types = avg_types.groupby(['cohort', 'type']) \
                     .agg({'rn':'mean'}).round().reset_index()
avg_types = avg_types.pivot_table(index='type',
                                  columns='cohort', values='rn')
avg_types

cohort,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06
type,,,,,,,,
Акция,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
Блог,4.0,3.0,2.0,2.0,2.0,2.0,2.0,1.0
Бонусы,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
Вебинар,3.0,2.0,3.0,3.0,8.0,2.0,2.0,2.0
Вебинар_апселл,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
Вебинары,1.0,1.0,1.0,1.0,1.0,1.0,1.0,<NA>
Главная,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
Интенсив,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0
Конференция,1.0,1.0,1.0,1.0,1.0,<NA>,1.0,<NA>


>**Комментарий**: *По когортам также видна разница в поведении пользователей, — отличия в кол-ве страниц блога. Блог яв-ся скорее всего ключевой страницей в знакомстве с порталом, задачами которые он может решить.*


### `16. Создадим функцию, показывающую среднее кол-во посещенных страниц по типу в зависимости от купленного продукта`<a id="step16"></a>

 [Вернуться в начало](#stepend)</n>

16.1 Создадим переменную с типом продукта по которому нужна информация


Типы продуктов - Трипвайер, Практикум, Вебинар, Подписка, Интенсив, Курс

In [84]:
product_type = "Курс"

16.2 Создадим функцию

In [85]:
def page_visits(x):
  page_visits = x.copy()
  page_visits[['1', 'num']] = page_visits['page_title'].str.split('#', expand=True)
  page_visits = page_visits[['ga_id','event_date','event_time', 'page_title','rang',
                             'type','rn', 'product_way', 'first_date',
                             'last_date', 'cohort', 'day_btw', 'num']]
  QUERY_4 = ('''
             SELECT *
             FROM `test29102023.logo.orders_for`
            ''')
  orders_for = client.query(QUERY_4).to_dataframe()

  orders_for['num'] = orders_for['num'].astype(str)

  page_visits = page_visits.merge(orders_for, how='left', on ='num')

  page_visits['earn'] = page_visits['earn'].fillna(0)
  page_visits['type_y'] = page_visits['type_y'].fillna('empty')
  page_visits['type_y'] = page_visits['type_y'].fillna('empty')
  page_visits = page_visits.rename(columns={'type_x': 'type_1'})
  page_visits = page_visits.rename(columns={'type_y': 'type_2'})

  page_visits['earn'] = page_visits['earn'].astype(int)

  page_visits_product = page_visits.copy()
  page_visits_product = page_visits_product.query('type_2 == @product_type')
  page_visits_product = page_visits_product[["ga_id"]]
  page_visits_product = page_visits_product.merge(page_visits, how="left", on="ga_id")

  avg_visits =  page_visits_product.copy()
  avg_visits = avg_visits.groupby(['type_1']) \
                         .agg({'rn':'mean'}) \
                         .rename(columns={'rn': 'avg_visits_page'}) \
                         .round() \
                         .sort_values('avg_visits_page', ascending=False) \
                         .reset_index()

  return avg_visits


In [86]:
page_visits(buyers)

,type_1,avg_visits_page
0,Блог,8.0
1,Вебинар,2.0
2,Интенсив,2.0
3,Курс,2.0
4,Лид-магнит,2.0
5,Мастер-класс,2.0
6,Акция,1.0
7,Бонусы,1.0
8,Вебинар_апселл,1.0
9,Главная,1.0


>**Комментарий**:
- Трипвайер - 3 Лид-магнита, 2 Блога, 2 Вебинара, 2 Интенсива, 2 Мастер-класса, 1 Подписка
- Вебинар -   1 Лид-магнит, 4 Блога, 3 Вебинара, 2 Интенсива, 3 Мастер-класса, 1 Подписка
- Практикум - 1 Лид-магнит, 2 Блога, 2 Вебинара, 4 Интенсива, 4 Мастер-класса, 3 Подписка
- Интенсив - 2 Лид-магнит, 2 Блога, 2 Вебинара, 1 Интенсив, 2 Мастер-класса, 1 Подписка
- Курс - 2 Лид-магнит, 8 Блога, 2 Вебинара, 2 Интенсив, 2 Мастер-класса, 1 Подписка


>**Комментарий**: Наблюдается, что перед покупкой Курса, пользователь посещает самое большое кол-во страниц блога. Связующие страницы — это лид-магнит и блог, скорее всего они яв-ся стартовыми для знакомства. Плюс, пользователь перед покупкой посещает все категории продуктов портала

### `17. Создадим функцию, показывающую среднее и медианное кол-во дней до покупки с разделением на когорты по месяцу первого визита`<a id="step17"></a>

 [Вернуться в начало](#stepend)</n>

17.1 Создадим переменную с типов продукта по которому нужна информация


In [87]:
product_type = "Вебинар"

17.2 Создадим функцию

In [88]:
def day_btw(x):
  page_visits = x.copy()
  page_visits[['1', 'num']] = page_visits['page_title'].str.split('#', expand=True)
  page_visits = page_visits[['ga_id','event_date','event_time', 'page_title','rang',
                             'type','rn', 'product_way', 'first_date',
                             'last_date', 'cohort', 'day_btw', 'num']]
  QUERY_4 = ('''
             SELECT *
             FROM `test29102023.logo.orders_for`
            ''')
  orders_for = client.query(QUERY_4).to_dataframe()

  orders_for['num'] = orders_for['num'].astype(str)

  page_visits = page_visits.merge(orders_for, how='left', on ='num')

  page_visits['earn'] = page_visits['earn'].fillna(0)
  page_visits['type_y'] = page_visits['type_y'].fillna('empty')
  page_visits['type_y'] = page_visits['type_y'].fillna('empty')
  page_visits = page_visits.rename(columns={'type_x': 'type_1'})
  page_visits = page_visits.rename(columns={'type_y': 'type_2'})

  page_visits['earn'] = page_visits['earn'].astype(int)

  page_visits_product = page_visits.copy()
  page_visits_product = page_visits_product.query('type_2 == @product_type')
  page_visits_product = page_visits_product[["ga_id"]]
  page_visits_product = page_visits_product.merge(page_visits, how="left", on="ga_id")

  avg_days = page_visits_product.copy()
  avg_days = avg_days.groupby('cohort') \
                     .agg({'day_btw':['mean','median']}) \
                     .round() \
                     .reset_index()
  return avg_days


In [89]:
day_btw(buyers)

cohort day_btw       
              mean median
0  2022-11  204.0   202.0
1  2023-01  150.0   158.0
2  2023-02  101.0   101.0
3  2023-03  80.0    81.0 
4  2023-04  45.0    45.0 
5  2023-05  22.0    23.0 
6  2023-06  1.0     0.0

>**Комментарий**:
- Трипвайер - Среднее кол-во дней до покупки 0-44
- Вебинар -   Среднее кол-во дней до покупки 1-22
- Практикум - Среднее кол-во дней до покупки 5-179
- Интенсив -  Среднее кол-во дней до покупки 1-19
- Курс - Среднее кол-во дней до покупки 2-18

### `18. Посмотрим путь пользователей`<a id="step18"></a>

 [Вернуться в начало](#stepend)</n>

18.1 Путь пользователя купившего Вебинар <a id="step18_1"></a>

 [Вернуться в начало](#stepend)</n>

In [90]:
product_type = "Вебинар"

In [91]:
def way(x):
  page_visits = x.copy()
  page_visits[['1', 'num']] = page_visits['page_title'].str.split('#', expand=True)
  page_visits = page_visits[['ga_id','event_date','event_time', 'page_title','rang',
                             'type','rn', 'product_way', 'first_date',
                             'last_date', 'cohort', 'day_btw', 'num']]
  QUERY_4 = ('''
             SELECT *
             FROM `test29102023.logo.orders_for`
            ''')
  orders_for = client.query(QUERY_4).to_dataframe()

  orders_for['num'] = orders_for['num'].astype(str)

  page_visits = page_visits.merge(orders_for, how='left', on ='num')

  page_visits['earn'] = page_visits['earn'].fillna(0)
  page_visits['type_y'] = page_visits['type_y'].fillna('empty')
  page_visits['type_y'] = page_visits['type_y'].fillna('empty')
  page_visits = page_visits.rename(columns={'type_x': 'type_1'})
  page_visits = page_visits.rename(columns={'type_y': 'type_2'})

  page_visits['earn'] = page_visits['earn'].astype(int)

  page_visits_product = page_visits.copy()
  page_visits_product = page_visits_product.query('type_2 == @product_type')
  page_visits_product = page_visits_product[["ga_id"]]
  page_visits_product = page_visits_product.merge(page_visits, how="left", on="ga_id")
  page_visits_product = page_visits_product.query('ga_id == "1346740381.161441477"')

  return page_visits_product


In [92]:
way(buyers)

,ga_id,event_date,event_time,page_title,rang,type_1,rn,product_way,first_date,last_date,cohort,day_btw,num,earn,type_2
249,1346740381.161441477,2022-11-19,2022-11-19 07:08:13,Лото. Посуда. Блог Лого-Эксперт,1,Блог,1,Блог1,2022-11-19,2023-06-09,2022-11,202,None,0,empty
250,1346740381.161441477,2022-11-21,2022-11-21 09:38:35,Картотека картинок-символов для работы над звукопроизношением. Блог Лого-Эксперт,2,Блог,2,Блог2,2022-11-19,2023-06-09,2022-11,202,None,0,empty
251,1346740381.161441477,2023-01-10,2023-01-10 13:07:59,Тяжелые нарушения речи (ТНР) в логопедии: коррекционная работа и занятия. Блог Лого-Эксперт,3,Блог,3,Блог3,2022-11-19,2023-06-09,2022-11,202,None,0,empty
252,1346740381.161441477,2023-01-26,2023-01-26 13:24:51,"Игра ""Дикие животные"". Блог Лого-Эксперт",4,Блог,4,Блог4,2022-11-19,2023-06-09,2022-11,202,None,0,empty
253,1346740381.161441477,2023-01-30,2023-01-30 09:02:13,Занятия Коняхиной. Предлоги. Блог Лого-Эксперт,5,Блог,5,Блог5,2022-11-19,2023-06-09,2022-11,202,None,0,empty
254,1346740381.161441477,2023-02-10,2023-02-10 09:03:55,"Дисграфия: виды, причины, профилактика и лечение. Блог Лого-Эксперт",6,Блог,6,Блог6,2022-11-19,2023-06-09,2022-11,202,None,0,empty
255,1346740381.161441477,2023-02-21,2023-02-21 05:28:45,Последовательность усвоения предлогов. Блог Лого-Эксперт,7,Блог,7,Блог7,2022-11-19,2023-06-09,2022-11,202,None,0,empty
256,1346740381.161441477,2023-03-26,2023-03-26 03:35:28,Игра «Один-много». Блог Лого-Эксперт,8,Блог,8,Блог8,2022-11-19,2023-06-09,2022-11,202,None,0,empty
257,1346740381.161441477,2023-04-05,2023-04-05 12:45:35,"Постановка звука Й в словах, слогах поэтапно. Блог Лого-Эксперт",9,Блог,9,Блог9,2022-11-19,2023-06-09,2022-11,202,None,0,empty
258,1346740381.161441477,2023-04-23,2023-04-23 08:47:34,Предлоги в картинках. Блог Лого-Эксперт,13,Блог,10,Блог10,2022-11-19,2023-06-09,2022-11,202,None,0,empty


18.2 Путь пользователя купившего Курс <a id="step18_2"></a>

 [Вернуться в начало](#stepend)</n>

In [93]:
product_type = "Курс"

In [94]:
def way(x):
  page_visits = x.copy()
  page_visits[['1', 'num']] = page_visits['page_title'].str.split('#', expand=True)
  page_visits = page_visits[['ga_id','event_date','event_time', 'page_title','rang',
                             'type','rn', 'product_way', 'first_date',
                             'last_date', 'cohort', 'day_btw', 'num']]
  QUERY_4 = ('''
             SELECT *
             FROM `test29102023.logo.orders_for`
            ''')
  orders_for = client.query(QUERY_4).to_dataframe()

  orders_for['num'] = orders_for['num'].astype(str)

  page_visits = page_visits.merge(orders_for, how='left', on ='num')

  page_visits['earn'] = page_visits['earn'].fillna(0)
  page_visits['type_y'] = page_visits['type_y'].fillna('empty')
  page_visits['type_y'] = page_visits['type_y'].fillna('empty')
  page_visits = page_visits.rename(columns={'type_x': 'type_1'})
  page_visits = page_visits.rename(columns={'type_y': 'type_2'})

  page_visits['earn'] = page_visits['earn'].astype(int)

  page_visits_product = page_visits.copy()
  page_visits_product = page_visits_product.query('type_2 == @product_type')
  page_visits_product = page_visits_product[["ga_id"]]
  page_visits_product = page_visits_product.merge(page_visits, how="left", on="ga_id")
  page_visits_product = page_visits_product.query('ga_id == "1274646780.168288852"')
  # page_visits_product = page_visits_product.drop(page_visits_product[page_visits_product.type_1 == "Оплата"].index)


  return page_visits_product


In [95]:
way(buyers)

,ga_id,event_date,event_time,page_title,rang,type_1,rn,product_way,first_date,last_date,cohort,day_btw,num,earn,type_2
188,1274646780.168288852,2023-04-30,2023-04-30 16:59:46,Экспресс-коррекция нарушений звукопроизношения,1,Мастер-класс,1,Мастер-класс1,2023-04-30,2023-06-21,2023-04,52,None,0,empty
189,1274646780.168288852,2023-04-30,2023-04-30 17:23:23,"Бесплатные материалы: вебинары, конспекты, рабочие тетради для логопедов | Дошкольная Академия",2,Вебинар,1,Вебинар1,2023-04-30,2023-06-21,2023-04,52,None,0,empty
190,1274646780.168288852,2023-05-01,2023-05-01 14:15:13,"Центр дистанционного и онлайн обучения и переподготовки логопедов ""Лого-Эксперт""",3,Главная,1,Главная1,2023-04-30,2023-06-21,2023-04,52,None,0,empty
191,1274646780.168288852,2023-05-01,2023-05-01 14:15:31,Практические материалы для работы логопедов и коррекционных специалистов,4,Курс,1,Курс1,2023-04-30,2023-06-21,2023-04,52,None,0,empty
192,1274646780.168288852,2023-05-01,2023-05-01 14:16:20,Мастер-классы для логопедов и корреляционных специалистов,5,Мастер-класс,2,Мастер-класс2,2023-04-30,2023-06-21,2023-04,52,None,0,empty
193,1274646780.168288852,2023-05-01,2023-05-01 14:16:34,О портале «Лого-Эксперт»,6,Блог,1,Блог1,2023-04-30,2023-06-21,2023-04,52,None,0,empty
194,1274646780.168288852,2023-05-01,2023-05-01 14:16:53,"Практикумы ""Давай заговорим""",7,Мастер-класс,3,Мастер-класс3,2023-04-30,2023-06-21,2023-04,52,None,0,empty
195,1274646780.168288852,2023-05-01,2023-05-01 14:18:35,Адаптация материалов - как это сделать?,9,Лид-магнит,1,Лид-магнит1,2023-04-30,2023-06-21,2023-04,52,None,0,empty
196,1274646780.168288852,2023-05-01,2023-05-01 14:22:48,"ИНТЕНСИВ Алалия: механизмы, диагностика и содержание логокоррекционных занятий",13,Интенсив,1,Интенсив1,2023-04-30,2023-06-21,2023-04,52,None,0,empty
197,1274646780.168288852,2023-05-01,2023-05-01 14:22:58,Диагностика и коррекция речевых нарушений в дошкольном возрасте,14,Мастер-класс,4,Мастер-класс4,2023-04-30,2023-06-21,2023-04,52,None,0,empty


>**Комментарий**: Важно провести исследование того, на какую работу нанимают каждый тип продукта. Из данных видно, хотя только из двух покупателей, у того кто купил Вебинар, сначала было изучение блога, затем страницы продуктов. А у того, кто купил Курс - изучение блога было перед покупкой. Скорее всего, тот кто купил Вебинар решал какую-то одну узкую задачу и пользователь искал ее решение, при этом искал решение бесплатно. Это как гипотеза исследований.



### `19. Пути пользователей до первой покупки с указанием продукта и кол-ва дней до покупки`<a id="step19"></a>

 [Вернуться в начало](#stepend)</n>

In [96]:
# 1 Создадим основную таблицу для анализа
test = buyers.copy()
# 2 Рассплитим колонку, чтобы достать номер заказа
test[['1', 'num']] = test['page_title'].str.split('#', expand=True)
# 3 Заполним пустые значения нулем
test['num'] = test['num'].fillna(0)
# 4 Оставим только нужные колонки
test = test[['ga_id', 'product_way', 'rang','num']]
# 5 Сформируем таблицу с данными по заказам
QUERY_5 = ('''
            SELECT *
            FROM `test29102023.logo.orders_for`
          ''')
orders_for_1 = client.query(QUERY_5).to_dataframe()
orders_for_1['num'] = orders_for_1['num'].astype(str)
# 6 Добавим к основной таблице данные заказов
test = test.merge(orders_for_1, how='left', on='num')
# 7 Заполним пропуски и преобразуем формат данных
test['earn'] = test['earn'].fillna(0)
test['type'] = test['type'].fillna('empty')
test['earn'] = test['earn'].astype(int)

# 8 Создадим таблицу, где только покупатели и соеденим ее с основной
test_1 = test.copy()
test_1 = test_1.query('earn !=0')
test_1 = test_1[['ga_id']]
test_1 = test_1.merge(test, how='left', on='ga_id')

# 8 Сцепим все страницы продуктов, которые посещал пользователей
test_1 = test_1.groupby('ga_id')['product_way'].agg(lambda x: x.tolist()).reset_index()
# 9 Переименуем столбец с путем пользователя для удобства
test_1 = test_1.rename(columns={"product_way": "path"})
# 10 Напишем функцию, вставляющую требуемый символ "">" между этапами пути пользователя
def listToString(test_1):
      str1 = ""
      for i in test_1['path']:

        str1 += i + " > "
      return str1[:-3]
# 11 Применим функцию к столбцу с путем пользователя
test_1['path'] = test_1.apply(listToString, axis=1)

# 12 Добавим к таблице данные по сумме заказа и кол-ве дней до оплаты
test_2 = test.copy()
test_2 = test_2.query('earn !=0')
test_2 = test_2[['ga_id','earn', 'type']]
test_1 = test_1.merge(test_2, how='left', on='ga_id')
test_3 = buyers.copy()
test_3 = test_3.query('type=="Оплата"')
test_3 = test_3[['ga_id', 'day_btw' ]]

# Создадим результирующую таблицу
test_1 = test_1.merge(test_3, how='left', on='ga_id')
pd.set_option('display.max_colwidth', -1)

ways = test_1.sort_values('earn', ascending=False).head(20)

<ipython-input-96-7a34ddd8a168>:54: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [97]:
ways

,ga_id,path,earn,type,day_btw
0,1014779213.168733044,Оплата1,19900,Курс,0
64,2091570019.167707081,Мастер-класс1 > Оплата1,19900,Курс,5
15,1274646780.168288852,Мастер-класс1 > Вебинар1 > Главная1 > Курс1 > Мастер-класс2 > Блог1 > Мастер-класс3 > Лид-магнит1 > Интенсив1 > Мастер-класс4 > Трипвайер1 > Подписка1 > Трипвайер2 > Вебинар2 > Трипвайер3 > Интенсив2 > Курс2 > Блог2 > Блог3 > Блог4 > Блог5 > Блог6 > Блог7 > Блог8 > Блог9 > Блог10 > Блог11 > Блог12 > Блог13 > Блог14 > Блог15 > Блог16 > Блог17 > Блог18 > Блог19 > Блог20 > Блог21 > Блог22 > Мастер-класс5 > Оплата1,19900,Курс,52
82,481725411.168511065,Мастер-класс1 > Акция1 > Трипвайер1 > Оплата1,19900,Курс,11
11,1139033893.168718250,Оплата1,18900,Курс,0
26,1421855575.166230368,Интенсив1 > Блог1 > Интенсив2 > Бонусы1 > Главная1 > Вебинар1 > Интенсив3 > Подписка1 > Лид-магнит1 > Лид-магнит2 > Лид-магнит3 > Трипвайер1 > Мастер-класс1 > Блог2 > Блог3 > Мастер-класс2 > Трипвайер2 > Оплата1,18900,Курс,227
80,439771058.167916933,Блог1 > Блог2 > Мастер-класс1 > Мастер-класс2 > Главная1 > Интенсив1 > Оплата1,9900,Интенсив,85
34,1525802365.167621475,Курс1 > Главная1 > Мастер-класс1 > Мастер-класс2 > Вебинар1 > Подписка1 > Вебинар2 > Мастер-класс3 > Вебинар3 > Вебинар4 > Мастер-класс4 > Акция1 > Вебинар5 > Интенсив1 > Курс2 > Оплата1,9900,Интенсив,114
40,1580386356.167887404,Главная1 > Акция1 > Курс1 > Интенсив1 > Оплата1,5900,Курс,85
60,2018372443.168613083,Главная1 > Акция1 > Интенсив1 > Оплата1,5900,Курс,0


>**Комментарий**: Из данных видно, что акции сокращают кол-во дней до покупки. Явно не наблюдается зависимость кол-ва дней до покупки от стоимости продукта. У большинства пользователей фигурируют Блог и посещение практически всех страниц продуктов. Т.е перед совершением покупки пользователь изучает весь спектр продуктов.



19.1 Рассчитаем среднее кол-во дней до покупки по типам продукта и общее среднее уже по пользователям купившим в июне<a id="step19_1"></a>

 [Вернуться в начало](#stepend)</n>

In [98]:
ways_avg_days = ways.copy()
ways_avg_days_products = ways_avg_days.groupby('type') \
                                     .agg({'day_btw':'mean'}) \
                                     .rename(columns={'day_btw': 'avg_days'}) \
                                     .round() \
                                     .reset_index()
ways_avg_days_products

,type,avg_days
0,Интенсив,101.0
1,Курс,28.0


19.2 Среднее кол-во дней до покупки у пользователей, купивших в июне<a id="step19_2"></a>

 [Вернуться в начало](#stepend)</n>

In [99]:
ways_avg_days = ways.copy()
ways_avg_days = ways_avg_days.day_btw.mean().round()
ways_avg_days

42.0

>**Комментарий**: Для рассчета затрат на привлечение новых платящих клиентов в продукт (CAC) возьму период в 60 дней. Получается, пример, что пользователи, которых мы привели в марте, купят в мае. Также, в период до покупки, как правило пользователь смотрит бесплатные материалы, лид-магниты, то следует раскинуть затраты и на "обслуживание" пользвоателя. Т.е в первый месяц его визита - маркетинговые затраты, во второй - обслуживание (скажем разделить затраты на инфраструктуру 50 на новых / 50 на старых) и в месяц покупки аналогичн разделить затраты на инфраструктуру.

>**Комментарий**: Использую вот такой рассчет CAC: (Затраты маркетинг в месяц первого визита + 1/2 затрат на инфраструктуру в следующий месяц + 1/2 затрат на инфраструктуру в месяц покупки / кол-во покупателей. Это будет наиболее приближенный CAC, его вполне можно использовать в Growth-процессах). Минусы - не учитываем сколько покупателей было из какой когорты - так как фактически, там могут быть и сделавшие покупку в первый визит, пользователи более старых когорт.

### `20. Сформируем данные для дальнейшего построения цепей Маркова`<a id="step20"></a>

 [Вернуться в начало](#stepend)</n>

>**Комментарий**: *Определим путь пользователей до первой покупки, используя марковскую модель, которая позволяет оценить важность каждой точки контакта и простроить на ее данных путь.*

20.1 Сформируем запрос и датафрейм с данными пользователей, которые совершили покупку<a id="step20_1"></a>

 [Вернуться в начало](#stepend)</n>

In [100]:
chain_conv = ('''
# 1. Таблица с пользователями с данными всех пользователей, результат таблицы visits
WITH users AS (SELECT ga_id,
                      event_date,
                      event_name,
                      event_time,
                      page_title,
                      RANK() OVER (PARTITION BY ga_id ORDER BY event_time ASC) AS rang,
                      utm_campaign,
                      utm_medium,
                      utm_source,
              FROM `test29102023.logo.visits_ga`
              WHERE ga_id IN (SELECT ga_id
                              FROM (SELECT ga_id,
                                          event_date,
                                          event_name,
                                          event_time,
                                          page_title,
                                          RANK() OVER (PARTITION BY ga_id ORDER BY event_time ASC) AS rang,
                                          utm_campaign,
                                          utm_medium,
                                          utm_source,
                                    FROM `test29102023.logo.visits_ga`
                                    WHERE (event_name = 'first_visit')
                                    AND (event_date >= '2022-11-04')) AS tab_2)),

# 2. Таблица с пользователями, у которых была хотя бы одна оплата
     buyers AS (SELECT *
                FROM `test29102023.logo.visits_ga`
                WHERE ga_id IN (SELECT ga_id
                                FROM (SELECT ga_id,
                                            SUM(b_status) AS b_status
                                      FROM (SELECT CAST(ga_id AS NUMERIC) AS ga_id,
                                                        event_date,
                                                        event_time,
                                                        page_title,
                                                        CAST((CASE
                                                          WHEN page_title LIKE '%Оплата%' THEN '1'
                                                          ELSE '0'
                                                        END) AS INT) AS b_status
                                                  FROM `test29102023.logo.visits_ga`
                                          ) AS tab_1
                                      GROUP BY ga_id
                                      HAVING b_status >= 1 ) AS tab_2)),

# 3. Таблица с пользователями, у которых не было покупок
      not_buyers AS (SELECT *
                  FROM `test29102023.logo.visits_ga`
                  WHERE ga_id IN (SELECT ga_id
                                  FROM (SELECT ga_id,
                                              SUM(b_status) AS b_status
                                        FROM (SELECT CAST(ga_id AS NUMERIC) AS ga_id,
                                                          event_date,
                                                          event_time,
                                                          page_title,
                                                          CAST((CASE
                                                            WHEN page_title LIKE '%Оплата%' THEN '1'
                                                            ELSE '0'
                                                          END) AS INT) AS b_status
                                                    FROM `test29102023.logo.visits_ga`
                                            ) AS tab_1
                                        GROUP BY ga_id
                                        HAVING b_status < 1 ) AS tab_2)),

# 4. Таблица, с указанием типа продукта по каждому названию
      type_products AS (SELECT string_field_0 AS page_title,
                             string_field_1 AS type
                       FROM `test29102023.logo.type_products`)


# 5. Таблица, в которой в отдельной колонке соединены тип продукта и его номер
SELECT *,
# Преобразуем значения под требования функции построения цепей - транслит и нумерацию преобразую в буквы
       CASE
          WHEN product_way  = "Блог1" THEN "BlogA"
          WHEN product_way  = "Блог2" THEN "BlogB"
          WHEN product_way  = "Блог3" THEN "BlogC"
          WHEN product_way  = "Блог4" THEN "BlogD"
          WHEN product_way  = "Блог5" THEN "BlogE"
          WHEN product_way  = "Блог6" THEN "BlogF"
          WHEN product_way  = "Блог7" THEN "BlogG"
          WHEN product_way  = "Блог8" THEN "BlogH"
          WHEN product_way  = "Блог9" THEN "BlogI"
          WHEN product_way  = "Блог10" THEN "BlogK"
          WHEN product_way  = "Блог11" THEN "BlogL"
          WHEN product_way  = "Блог12" THEN "BlogL"
          WHEN product_way  = "Блог13" THEN "BlogL"
          WHEN product_way  = "Блог14" THEN "BlogL"
          WHEN product_way  = "Блог15" THEN "BlogL"
          WHEN product_way  = "Блог16" THEN "BlogL"
          WHEN product_way  = "Блог17" THEN "BlogL"
          WHEN product_way  = "Блог18" THEN "BlogL"
          WHEN product_way  = "Блог19" THEN "BlogL"
          WHEN product_way  = "Блог20" THEN "BlogL"
          WHEN product_way  = "Блог21" THEN "BlogL"
          WHEN product_way  = "Блог22" THEN "BlogL"
          WHEN product_way  = "Блог23" THEN "BlogL"
          WHEN product_way  = "Блог24" THEN "BlogL"
          WHEN product_way  = "Блог25" THEN "BlogL"
          WHEN product_way  = "Блог26" THEN "BlogL"
          WHEN product_way  = "Блог27" THEN "BlogL"
          WHEN product_way  = "Блог28" THEN "BlogL"
          WHEN product_way  = "Блог29" THEN "BlogL"
          WHEN product_way  = "Блог30" THEN "BlogL"
          WHEN product_way  = "Блог31" THEN "BlogL"
          WHEN product_way  = "Блог32" THEN "BlogL"
          WHEN product_way  = "Блог33" THEN "BlogL"
          WHEN product_way  = "Блог34" THEN "BlogL"
          WHEN product_way  = "Блог35" THEN "BlogL"
          WHEN product_way  = "Блог36" THEN "BlogL"
          WHEN product_way  = "Блог37" THEN "BlogL"
          WHEN product_way  = "Блог38" THEN "BlogL"
          WHEN product_way  = "Блог39" THEN "BlogL"
          WHEN product_way  = "Блог40" THEN "BlogL"
          WHEN product_way  = "Блог41" THEN "BlogL"
          WHEN product_way  = "Блог42" THEN "BlogL"
          WHEN product_way  = "Блог43" THEN "BlogL"
          WHEN product_way  = "Блог44" THEN "BlogL"
          WHEN product_way  = "Блог45" THEN "BlogL"
          WHEN product_way  = "Блог46" THEN "BlogL"
          WHEN product_way  = "Блог47" THEN "BlogL"
          WHEN product_way  = "Блог48" THEN "BlogL"
          WHEN product_way  = "Блог49" THEN "BlogL"
          WHEN product_way  = "Блог50" THEN "BlogL"
          WHEN product_way  = "Блог51" THEN "BlogL"
          WHEN product_way  = "Блог52" THEN "BlogL"
          WHEN product_way  = "Блог53" THEN "BlogL"
          WHEN product_way  = "Блог54" THEN "BlogL"
          WHEN product_way  = "Блог55" THEN "BlogL"
          WHEN product_way  = "Блог56" THEN "BlogL"
          WHEN product_way  = "Блог57" THEN "BlogL"
          WHEN product_way  = "Блог58" THEN "BlogL"
          WHEN product_way  = "Блог59" THEN "BlogL"
          WHEN product_way  = "Блог60" THEN "BlogL"
          WHEN product_way  = "Блог61" THEN "BlogL"
          WHEN product_way  = "Блог62" THEN "BlogL"
          WHEN product_way  = "Блог63" THEN "BlogL"
          WHEN product_way  = "Блог64" THEN "BlogL"
          WHEN product_way  = "Блог65" THEN "BlogL"
          WHEN product_way  = "Блог66" THEN "BlogL"
          WHEN product_way  = "Блог67" THEN "BlogL"
          WHEN product_way  = "Блог68" THEN "BlogL"
          WHEN product_way  = "Блог69" THEN "BlogL"
          WHEN product_way  = "Блог70" THEN "BlogL"
          WHEN product_way  = "Блог71" THEN "BlogL"
          WHEN product_way  = "Блог72" THEN "BlogL"
          WHEN product_way  = "Блог73" THEN "BlogL"
          WHEN product_way  = "Блог74" THEN "BlogL"
          WHEN product_way  = "Блог75" THEN "BlogL"
          WHEN product_way  = "Блог76" THEN "BlogL"
          WHEN product_way  = "Блог77" THEN "BlogL"
          WHEN product_way  = "Блог78" THEN "BlogL"
          WHEN product_way  = "Блог79" THEN "BlogL"
          WHEN product_way  = "Вебинар1" THEN "WebA"
          WHEN product_way  = "Вебинар2" THEN "WebB"
          WHEN product_way  = "Вебинар3" THEN "WebC"
          WHEN product_way  = "Вебинар4" THEN "WebD"
          WHEN product_way  = "Вебинар5" THEN "WebE"
          WHEN product_way  = "Вебинар6" THEN "WebF"
          WHEN product_way  = "Вебинар7" THEN "WebG"
          WHEN product_way  = "Вебинар8" THEN "WebH"
          WHEN product_way  = "Вебинар9" THEN "WebI"
          WHEN product_way  = "Вебинар10" THEN "WebK"
          WHEN product_way  = "Вебинар11" THEN "WebL"
          WHEN product_way  = "Вебинар12" THEN "WebL"
          WHEN product_way  = "Вебинар13" THEN "WebL"
          WHEN product_way  = "Вебинар14" THEN "WebL"
          WHEN product_way  = "Вебинар15" THEN "WebL"
          WHEN product_way  = "Вебинар16" THEN "WebL"
          WHEN product_way  = "Вебинар17" THEN "WebL"
          WHEN product_way  = "Вебинар18" THEN "WebL"
          WHEN product_way  = "Вебинар19" THEN "WebL"
          WHEN product_way  = "Вебинар20" THEN "WebL"
          WHEN product_way  = "Вебинар21" THEN "WebL"
          WHEN product_way  = "Вебинар22" THEN "WebL"
          WHEN product_way  = "Вебинар23" THEN "WebL"
          WHEN product_way  = "Вебинар24" THEN "WebL"
          WHEN product_way  = "Вебинар25" THEN "WebL"
          WHEN product_way  = "Вебинар26" THEN "WebL"
          WHEN product_way  = "Вебинар27" THEN "WebL"
          WHEN product_way  = "Вебинар28" THEN "WebL"
          WHEN product_way  = "Вебинар29" THEN "WebL"
          WHEN product_way  = "Вебинар30" THEN "WebL"
          WHEN product_way  = "Вебинар31" THEN "WebL"
          WHEN product_way  = "Вебинар32" THEN "WebL"
          WHEN product_way  = "Вебинар33" THEN "WebL"
          WHEN product_way  = "Вебинар34" THEN "WebL"
          WHEN product_way  = "Вебинар35" THEN "WebL"
          WHEN product_way  = "Вебинар36" THEN "WebL"
          WHEN product_way  = "Вебинар37" THEN "WebL"
          WHEN product_way  = "Вебинар38" THEN "WebL"
          WHEN product_way  = "Вебинар39" THEN "WebL"
          WHEN product_way  = "Вебинар40" THEN "WebL"
          WHEN product_way  = "Вебинар41" THEN "WebL"
          WHEN product_way  = "Вебинар42" THEN "WebL"
          WHEN product_way  = "Вебинар43" THEN "WebL"
          WHEN product_way  = "Вебинар44" THEN "WebL"
          WHEN product_way  = "Вебинар45" THEN "WebL"
          WHEN product_way  = "Вебинар46" THEN "WebL"
          WHEN product_way  = "Вебинар47" THEN "WebL"
          WHEN product_way  = "Вебинар48" THEN "WebL"
          WHEN product_way  = "Вебинар49" THEN "WebL"
          WHEN product_way  = "Вебинар50" THEN "WebL"
          WHEN product_way  = "Вебинар51" THEN "WebL"
          WHEN product_way  = "Вебинар52" THEN "WebL"
          WHEN product_way  = "Вебинар53" THEN "WebL"
          WHEN product_way  = "Вебинар54" THEN "WebL"
          WHEN product_way  = "Вебинар55" THEN "WebL"
          WHEN product_way  = "Вебинар56" THEN "WebL"
          WHEN product_way  = "Вебинар57" THEN "WebL"
          WHEN product_way  = "Вебинар58" THEN "WebL"
          WHEN product_way  = "Вебинар59" THEN "WebL"
          WHEN product_way  = "Вебинар60" THEN "WebL"
          WHEN product_way  = "Вебинар61" THEN "WebL"
          WHEN product_way  = "Вебинар62" THEN "WebL"
          WHEN product_way  = "Вебинар63" THEN "WebL"
          WHEN product_way  = "Вебинар64" THEN "WebL"
          WHEN product_way  = "Вебинар65" THEN "WebL"
          WHEN product_way  = "Вебинар66" THEN "WebL"
          WHEN product_way  = "Вебинар67" THEN "WebL"
          WHEN product_way  = "Вебинар68" THEN "WebL"
          WHEN product_way  = "Вебинар69" THEN "WebL"
          WHEN product_way  = "Вебинар70" THEN "WebL"
          WHEN product_way  = "Вебинар71" THEN "WebL"
          WHEN product_way  = "Вебинар72" THEN "WebL"
          WHEN product_way  = "Вебинар73" THEN "WebL"
          WHEN product_way  = "Вебинар74" THEN "WebL"
          WHEN product_way  = "Вебинар75" THEN "WebL"
          WHEN product_way  = "Вебинар76" THEN "WebL"
          WHEN product_way  = "Вебинар77" THEN "WebL"
          WHEN product_way  = "Вебинар78" THEN "WebL"
          WHEN product_way  = "Вебинар79" THEN "WebL"
          WHEN product_way  = "Вебинар80" THEN "WebL"
          WHEN product_way  = "Вебинар81" THEN "WebL"
          WHEN product_way  = "Вебинар82" THEN "WebL"
          WHEN product_way  = "Вебинар83" THEN "WebL"
          WHEN product_way  = "Вебинар84" THEN "WebL"
          WHEN product_way  = "Вебинар85" THEN "WebL"
          WHEN product_way  = "Вебинар86" THEN "WebL"
          WHEN product_way  = "Вебинар87" THEN "WebL"
          WHEN product_way  = "Вебинар88" THEN "WebL"
          WHEN product_way  = "Вебинар89" THEN "WebL"
          WHEN product_way  = "Вебинар90" THEN "WebL"
          WHEN product_way  = "Вебинар91" THEN "WebL"
          WHEN product_way  = "Вебинар92" THEN "WebL"
          WHEN product_way  = "Вебинар93" THEN "WebL"
          WHEN product_way  = "Вебинар94" THEN "WebL"
          WHEN product_way  = "Вебинар95" THEN "WebL"
          WHEN product_way  = "Вебинар96" THEN "WebL"
          WHEN product_way  = "Вебинар97" THEN "WebL"
          WHEN product_way  = "Вебинар98" THEN "WebL"
          WHEN product_way  = "Вебинар_апселл1" THEN "WebL"
          WHEN product_way  = "Вебинар_апселл2" THEN "WebL"
          WHEN product_way  = "Заявки на вебинар1" THEN "WebL"
          WHEN product_way  = "Заявки на вебинар2" THEN "WebL"
          WHEN product_way  = "Заявки на вебинар3" THEN "WebL"
          WHEN product_way  = "Интенсив1" THEN "IntA"
          WHEN product_way  = "Интенсив2" THEN "IntB"
          WHEN product_way  = "Интенсив3" THEN "IntC"
          WHEN product_way  = "Интенсив4" THEN "IntD"
          WHEN product_way  = "Интенсив5" THEN "IntE"
          WHEN product_way  = "Интенсив6" THEN "IntF"
          WHEN product_way  = "Интенсив7" THEN "IntG"
          WHEN product_way  = "Интенсив8" THEN "IntH"
          WHEN product_way  = "Интенсив9" THEN "IntI"
          WHEN product_way  = "Интенсив10" THEN "IntK"
          WHEN product_way  = "Интенсив11" THEN "IntL"
          WHEN product_way  = "Интенсив12" THEN "IntL"
          WHEN product_way  = "Интенсив13" THEN "IntL"
          WHEN product_way  = "Интенсив14" THEN "IntL"
          WHEN product_way  = "Интенсив15" THEN "IntL"
          WHEN product_way  = "Интенсив16" THEN "IntL"
          WHEN product_way  = "Интенсив17" THEN "IntL"
          WHEN product_way  = "Интенсив18" THEN "IntL"
          WHEN product_way  = "Интенсив19" THEN "IntL"
          WHEN product_way  = "Интенсив20" THEN "IntL"
          WHEN product_way  = "Интенсив21" THEN "IntL"
          WHEN product_way  = "Интенсив22" THEN "IntL"
          WHEN product_way  = "Интенсив23" THEN "IntL"
          WHEN product_way  = "Интенсив24" THEN "IntL"
          WHEN product_way  = "Интенсив25" THEN "IntL"
          WHEN product_way  = "Интенсив26" THEN "IntL"
          WHEN product_way  = "Интенсив27" THEN "IntL"
          WHEN product_way  = "Интенсив28" THEN "IntL"
          WHEN product_way  = "Интенсив29" THEN "IntL"
          WHEN product_way  = "Интенсив30" THEN "IntL"
          WHEN product_way  = "Интенсив31" THEN "IntL"
          WHEN product_way  = "Интенсив32" THEN "IntL"
          WHEN product_way  = "Интенсив33" THEN "IntL"
          WHEN product_way  = "Интенсив34" THEN "IntL"
          WHEN product_way  = "Интенсив35" THEN "IntL"
          WHEN product_way  = "Интенсив36" THEN "IntL"
          WHEN product_way  = "Интенсив37" THEN "IntL"
          WHEN product_way  = "Интенсив38" THEN "IntL"
          WHEN product_way  = "Интенсив39" THEN "IntL"
          WHEN product_way  = "Интенсив40" THEN "IntL"
          WHEN product_way  = "Интенсив41" THEN "IntL"
          WHEN product_way  = "Интенсив42" THEN "IntL"
          WHEN product_way  = "Интенсив43" THEN "IntL"
          WHEN product_way  = "Интенсив44" THEN "IntL"
          WHEN product_way  = "Интенсив45" THEN "IntL"
          WHEN product_way  = "Интенсив46" THEN "IntL"
          WHEN product_way  = "Интенсив47" THEN "IntL"
          WHEN product_way  = "Интенсив48" THEN "IntL"
          WHEN product_way  = "Интенсив49" THEN "IntL"
          WHEN product_way  = "Курс1" THEN "KursA"
          WHEN product_way  = "Курс2" THEN "KursB"
          WHEN product_way  = "Курс3" THEN "KursC"
          WHEN product_way  = "Курс4" THEN "KursD"
          WHEN product_way  = "Курс5" THEN "KursE"
          WHEN product_way  = "Курс6" THEN "KursF"
          WHEN product_way  = "Курс7" THEN "KursG"
          WHEN product_way  = "Курс8" THEN "KursH"
          WHEN product_way  = "Курс9" THEN "KursI"
          WHEN product_way  = "Курс10" THEN "KursK"
          WHEN product_way  = "Курс11" THEN "KursL"
          WHEN product_way  = "Курс12" THEN "KursL"
          WHEN product_way  = "Курс13" THEN "KursL"
          WHEN product_way  = "Курс14" THEN "KursL"
          WHEN product_way  = "Курс15" THEN "KursL"
          WHEN product_way  = "Курс16" THEN "KursL"
          WHEN product_way  = "Курс17" THEN "KursL"
          WHEN product_way  = "Курс18" THEN "KursL"
          WHEN product_way  = "Курс19" THEN "KursL"
          WHEN product_way  = "Курс20" THEN "KursL"
          WHEN product_way  = "Курс21" THEN "KursL"
          WHEN product_way  = "Курс22" THEN "KursL"
          WHEN product_way  = "Курс23" THEN "KursL"
          WHEN product_way  = "Курс24" THEN "KursL"
          WHEN product_way  = "Курс25" THEN "KursL"
          WHEN product_way  = "Курс26" THEN "KursL"
          WHEN product_way  = "Курс27" THEN "KursL"
          WHEN product_way  = "Курс28" THEN "KursL"
          WHEN product_way  = "Курс29" THEN "KursL"
          WHEN product_way  = "Курс30" THEN "KursL"
          WHEN product_way  = "Курс31" THEN "KursL"
          WHEN product_way  = "Курс32" THEN "KursL"
          WHEN product_way  = "Курс33" THEN "KursL"
          WHEN product_way  = "Лид-магнит1" THEN "LmA"
          WHEN product_way  = "Лид-магнит2" THEN "LmB"
          WHEN product_way  = "Лид-магнит3" THEN "LmC"
          WHEN product_way  = "Лид-магнит4" THEN "LmD"
          WHEN product_way  = "Лид-магнит5" THEN "LmE"
          WHEN product_way  = "Лид-магнит6" THEN "LmF"
          WHEN product_way  = "Лид-магнит7" THEN "LmG"
          WHEN product_way  = "Лид-магнит8" THEN "LmH"
          WHEN product_way  = "Лид-магнит9" THEN "LmI"
          WHEN product_way  = "Лид-магнит10" THEN "LmK"
          WHEN product_way  = "Лид-магнит11" THEN "LmL"
          WHEN product_way  = "Лид-магнит12" THEN "LmL"
          WHEN product_way  = "Лид-магнит13" THEN "LmL"
          WHEN product_way  = "Лид-магнит14" THEN "LmL"
          WHEN product_way  = "Лид-магнит15" THEN "LmL"
          WHEN product_way  = "Лид-магнит16" THEN "LmL"
          WHEN product_way  = "Лид-магнит17" THEN "LmL"
          WHEN product_way  = "Лид-магнит18" THEN "LmL"
          WHEN product_way  = "Мастер-класс1" THEN "MkA"
          WHEN product_way  = "Мастер-класс2" THEN "MkB"
          WHEN product_way  = "Мастер-класс3" THEN "MkC"
          WHEN product_way  = "Мастер-класс4" THEN "MkD"
          WHEN product_way  = "Мастер-класс5" THEN "MkE"
          WHEN product_way  = "Мастер-класс6" THEN "MkF"
          WHEN product_way  = "Мастер-класс7" THEN "MkG"
          WHEN product_way  = "Мастер-класс8" THEN "MkH"
          WHEN product_way  = "Мастер-класс9" THEN "MkI"
          WHEN product_way  = "Мастер-класс10" THEN "MkK"
          WHEN product_way  = "Мастер-класс11" THEN "MkL"
          WHEN product_way  = "Мастер-класс12" THEN "MkL"
          WHEN product_way  = "Мастер-класс13" THEN "MkL"
          WHEN product_way  = "Мастер-класс14" THEN "MkL"
          WHEN product_way  = "Мастер-класс15" THEN "MkL"
          WHEN product_way  = "Мастер-класс16" THEN "MkL"
          WHEN product_way  = "Мастер-класс17" THEN "MkL"
          WHEN product_way  = "Мастер-класс18" THEN "MkL"
          WHEN product_way  = "Мастер-класс19" THEN "MkL"
          WHEN product_way  = "Мастер-класс20" THEN "MkL"
          WHEN product_way  = "Мастер-класс21" THEN "MkL"
          WHEN product_way  = "Мастер-класс22" THEN "MkL"
          WHEN product_way  = "Мастер-класс23" THEN "MkL"
          WHEN product_way  = "Мастер-класс24" THEN "MkL"
          WHEN product_way  = "Мастер-класс25" THEN "MkL"
          WHEN product_way  = "Мастер-класс26" THEN "MkL"
          WHEN product_way  = "Мастер-класс27" THEN "MkL"
          WHEN product_way  = "Мастер-класс28" THEN "MkL"
          WHEN product_way  = "Мастер-класс29" THEN "MkL"
          WHEN product_way  = "Мастер-класс30" THEN "MkL"
          WHEN product_way  = "Мастер-класс31" THEN "MkL"
          WHEN product_way  = "Мастер-класс32" THEN "MkL"
          WHEN product_way  = "Мастер-класс33" THEN "MkL"
          WHEN product_way  = "Мастер-класс34" THEN "MkL"
          WHEN product_way  = "Мастер-класс35" THEN "MkL"
          WHEN product_way  = "Мастер-класс36" THEN "MkL"
          WHEN product_way  = "Мастер-класс37" THEN "MkL"
          WHEN product_way  = "Мастер-класс38" THEN "MkL"
          WHEN product_way  = "Мастер-класс39" THEN "MkL"
          WHEN product_way  = "Мастер-класс40" THEN "MkL"
          WHEN product_way  = "Мастер-класс41" THEN "MkL"
          WHEN product_way  = "Мастер-класс42" THEN "MkL"
          WHEN product_way  = "Мастер-класс43" THEN "MkL"
          WHEN product_way  = "Мастер-класс44" THEN "MkL"
          WHEN product_way  = "Мастер-класс45" THEN "MkL"
          WHEN product_way  = "Мастер-класс46" THEN "MkL"
          WHEN product_way  = "Мастер-класс47" THEN "MkL"
          WHEN product_way  = "Мастер-класс48" THEN "MkL"
          WHEN product_way  = "Мастер-класс49" THEN "MkL"
          WHEN product_way  = "Мастер-класс50" THEN "MkL"
          WHEN product_way  = "Мастер-класс51" THEN "MkL"
          WHEN product_way  = "Мастер-класс52" THEN "MkL"
          WHEN product_way  = "Мастер-класс53" THEN "MkL"
          WHEN product_way  = "Мастер-класс54" THEN "MkL"
          WHEN product_way  = "Мастер-класс55" THEN "MkL"
          WHEN product_way  = "Мастер-класс56" THEN "MkL"
          WHEN product_way  = "Мастер-класс57" THEN "MkL"
          WHEN product_way  = "Нескучная логопедия1" THEN "FunA"
          WHEN product_way  = "Нескучная логопедия2" THEN "FunB"
          WHEN product_way  = "Нескучная логопедия3" THEN "FunC"
          WHEN product_way  = "Нескучная логопедия4" THEN "FunD"
          WHEN product_way  = "Нескучная логопедия5" THEN "FunE"
          WHEN product_way  = "Нескучная логопедия6" THEN "FunF"
          WHEN product_way  = "Нескучная логопедия7" THEN "FunG"
          WHEN product_way  = "Нескучная логопедия8" THEN "FunH"
          WHEN product_way  = "Нескучная логопедия9" THEN "FunI"
          WHEN product_way  = "Нескучная логопедия10" THEN "FunK"
          WHEN product_way  = "Нескучная логопедия11" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия12" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия13" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия14" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия15" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия16" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия17" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия18" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия19" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия20" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия21" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия22" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия23" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия24" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия25" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия26" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия27" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия28" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия29" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия30" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия31" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия32" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия33" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия34" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия35" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия36" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия37" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия38" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия39" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия40" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия41" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия42" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия43" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия44" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия45" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия46" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия47" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия48" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия49" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия50" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия51" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия52" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия53" THEN "FunL"
          WHEN product_way  = "Логошоу1" THEN "ShowA"
          WHEN product_way  = "Логошоу2" THEN "ShowB"
          WHEN product_way  = "Подписка1" THEN "SubsA"
          WHEN product_way  = "Подписка2" THEN "SubsB"
          WHEN product_way  = "Подписка3" THEN "SubsC"
          WHEN product_way  = "Подписка4" THEN "SubsD"
          WHEN product_way  = "Подписка5" THEN "SubsE"
          WHEN product_way  = "Подписка6" THEN "SubsF"
          WHEN product_way  = "Подписка7" THEN "SubsG"
          WHEN product_way  = "Подписка8" THEN "SubsH"
          WHEN product_way  = "Подписка9" THEN "SubsI"
          WHEN product_way  = "Подписка10" THEN "SubsK"
          WHEN product_way  = "Подписка11" THEN "SubsL"
          WHEN product_way  = "Подписка12" THEN "SubsL"
          WHEN product_way  = "Подписка13" THEN "SubsL"
          WHEN product_way  = "Подписка14" THEN "SubsL"
          WHEN product_way  = "Подписка15" THEN "SubsL"
          WHEN product_way  = "Подписка16" THEN "SubsL"
          WHEN product_way  = "Подписка17" THEN "SubsL"
          WHEN product_way  = "Подписка18" THEN "SubsL"
          WHEN product_way  = "Подписка19" THEN "SubsL"
          WHEN product_way  = "Рабочая программа1" THEN "WorkA"
          WHEN product_way  = "Рабочая программа2" THEN "WorkB"
          WHEN product_way  = "Рабочая программа3" THEN "WorkC"
          WHEN product_way  = "Рабочая программа4" THEN "WorkD"
          WHEN product_way  = "Супервизия1" THEN "SuperA"
          WHEN product_way  = "Супервизия2" THEN "SuperB"
          WHEN product_way  = "Трипвайер1" THEN "TripA"
          WHEN product_way  = "Трипвайер2" THEN "TripB"
          WHEN product_way  = "Трипвайер3" THEN "TripC"
          WHEN product_way  = "Трипвайер4" THEN "TripD"
          WHEN product_way  = "Трипвайер5" THEN "TripE"
          WHEN product_way  = "Трипвайер6" THEN "TripF"
          WHEN product_way  = "Трипвайер7" THEN "TripG"
          WHEN product_way  = "Трипвайер8" THEN "TripH"
          WHEN product_way  = "Трипвайер9" THEN "TripI"
          WHEN product_way  = "Трипвайер10" THEN "TripK"
          WHEN product_way  = "Трипвайер11" THEN "TripL"
          WHEN product_way  = "Трипвайер12" THEN "TripL"
          WHEN product_way  = "Трипвайер13" THEN "TripL"
          WHEN product_way  = "Трипвайер14" THEN "TripL"
          WHEN product_way  = "Трипвайер15" THEN "TripL"
          WHEN product_way  = "Трипвайер16" THEN "TripL"
          WHEN product_way  = "Трипвайер17" THEN "TripL"
          WHEN product_way  = "Трипвайер18" THEN "TripL"
          WHEN product_way  = "Трипвайер19" THEN "TripL"
          WHEN product_way  = "Трипвайер20" THEN "TripL"
          WHEN product_way  = "Трипвайер21" THEN "TripL"
          WHEN product_way  = "Трипвайер22" THEN "TripL"
          WHEN product_way  = "Трипвайер23" THEN "TripL"
          WHEN product_way  = "Трипвайер24" THEN "TripL"
          WHEN product_way  = "Трипвайер25" THEN "TripL"
          WHEN product_way  = "Трипвайер26" THEN "TripL"
          WHEN product_way  = "Трипвайер27" THEN "TripL"
          WHEN product_way  = "Трипвайер28" THEN "TripL"
          WHEN product_way  = "Трипвайер29" THEN "TripL"
          WHEN product_way  = "Трипвайер30" THEN "TripL"
          WHEN product_way  = "Трипвайер31" THEN "TripL"
          WHEN product_way  = "Трипвайер32" THEN "TripL"
          WHEN product_way  = "Трипвайер33" THEN "TripL"
          WHEN product_way  = "Трипвайер34" THEN "TripL"
          WHEN product_way  = "Трипвайер35" THEN "TripL"
          WHEN product_way  = "Трипвайер36" THEN "TripL"
          WHEN product_way  = "Оплата2" THEN "BuyA"
          WHEN product_way  = "Оплата3" THEN "BuyB"
          WHEN product_way  = "Конференция1" THEN "ConfA"
          WHEN product_way  = "Акция1" THEN "AkciaA"
          WHEN product_way  = "Акция2" THEN "AkciaB"
          WHEN product_way  = "Акция3" THEN "AkciaC"
          WHEN product_way  = "Бонусы1" THEN "BonusA"
          WHEN product_way  = "Главная1" THEN "MainA"
          WHEN product_way  = "Главная2" THEN "MainB"
       END AS product_type
FROM (SELECT ga_id,
            event_date,
            event_time,
            event_name,
            page_title,
            rang,
            type,
            rn,
            CONCAT(type, '', rn) AS product_way,
      FROM (  # 5.1 Таблица, в которой проранжированы посещения страниц в рамках каждого типа продукта
              SELECT *,
                    ROW_NUMBER() over (partition by ga_id, type order by event_time) as rn,
                    CONCAT(utm_campaign, '_', utm_medium, '_', utm_source) AS traffic_way
              FROM (# 5.2 Таблица, в которой убраны все строки, которые идут после первой оплаты + добавлена колонка типа продукта
                    SELECT tab_1.ga_id AS ga_id, # Уникальный идентификатор пользователя в GA
                          tab_1.event_date AS event_date, # Дата визита
                          tab_1.event_name AS event_name, # Характеристика визита
                          tab_1.event_time AS event_time, # Дата и время визита
                          tab_1.page_title AS page_title, # Название страницы визита
                          tab_1.rang AS rang, # Номер страницы посещения в рамках пользователя, нумерация каждого типа продукта, ранжирование по event_time ASC
                          tab_1.utm_campaign AS utm_campaign, # метка utm_campaign
                          tab_1.utm_medium AS utm_medium, # метка utm_medium
                          tab_1.utm_source AS utm_source, # метка utm_source
                          tab_3.type AS type # тип продукта
                    FROM users AS tab_1
                    INNER JOIN (# 5.3 Таблица, в sm начинается нумерация с с первой оплаты по каждому пользователю и исключает пользователей к которых
                                SELECT ga_id,
                                      event_date,
                                      event_name,
                                      event_time,
                                      page_title,
                                      rang,
                                      sm
                                FROM (SELECT ga_id,
                                            event_date,
                                            event_name,
                                            event_time,
                                            page_title,
                                            RANK() OVER (PARTITION BY ga_id ORDER BY event_time ASC) AS rang,
                                            SUM(CASE WHEN page_title LIKE '%Оплата%' THEN 1 ELSE 0 END) OVER (PARTITION BY ga_id ORDER BY event_time) AS sm # суммируем покупки по окну со строкой "Оплата"
                                      FROM `test29102023.logo.visits_ga`) AS tab_2
                                --WHERE ga_id = 10603501.167690103
                                )
                    AS tab_2 ON tab_2.ga_id = tab_1.ga_id AND tab_1.rang <= tab_2.rang

                    LEFT JOIN type_products AS tab_3 ON tab_3.page_title = tab_1.page_title
                    # В следующей строке условие tab_2.sm = 1 исключает пользователей без оплаты (если поставить =2, то будет путь пользователя до второй оплаты)
                    WHERE tab_2.sm = 1 AND tab_2.page_title LIKE '%Оплата%') AS tab_4
              ORDER BY rang ASC) AS tab_5
        WHERE (page_title != 'None')
        AND (tab_5.type != 'Техническое')
        AND (tab_5.type != 'Тестирование')
        AND (tab_5.type != 'Удостоверение')
        AND (tab_5.type != 'Заявка')
        AND (tab_5.type != 'Заказ')
        AND (page_title != 'None')
        AND (tab_5.type != 'Покупка')
        AND (tab_5.type != 'Лекция')
      ) AS main_tab
''')

In [101]:
buyers_chain = client.query(chain_conv).to_dataframe()
buyers_chain.head(5)

,ga_id,event_date,event_time,event_name,page_title,rang,type,rn,product_way,product_type
0,2136012.168544892,2023-05-30,2023-05-30 12:15:22+00:00,first_visit,«Структура и содержание индивидуального логопедического занятия в ДОО»,1,Интенсив,1,Интенсив1,IntA
1,14268773.160667584,2023-04-16,2023-04-16 05:03:08+00:00,first_visit,Чемоданчик логопеда | Дошкольная Академия,1,Мастер-класс,1,Мастер-класс1,MkA
2,26565126.167655439,2023-02-16,2023-02-16 13:33:12+00:00,first_visit,Подборка коллажей с 1 по 8 тип слоговой структуры слова. Блог Лого-Эксперт,1,Блог,1,Блог1,BlogA
3,57799351.158624927,2022-11-20,2022-11-20 06:05:28+00:00,first_visit,Почему ребёнок не понимает речь? Причины и способы коррекции,1,Трипвайер,1,Трипвайер1,TripA
4,81880761.161656716,2022-12-05,2022-12-05 17:30:08+00:00,first_visit,"Центр дистанционного и онлайн обучения и переподготовки логопедов ""Лого-Эксперт""",1,Главная,1,Главная1,MainA


20.2 Сформируем запрос и датафрейм с данными пользователей, которые не совершили покупку<a id="step20_2"></a>

 [Вернуться в начало](#stepend)</n>

In [102]:
chain_no_conv = ('''
# 1. Таблица с пользователями с данными всех пользователей, результат таблицы visits
WITH users AS (SELECT ga_id,
                      event_date,
                      event_name,
                      event_time,
                      page_title,
                      RANK() OVER (PARTITION BY ga_id ORDER BY event_time ASC) AS rang,
                      utm_campaign,
                      utm_medium,
                      utm_source,
              FROM `test29102023.logo.visits_ga`
              WHERE ga_id IN (SELECT ga_id
                              FROM (SELECT ga_id,
                                          event_date,
                                          event_name,
                                          event_time,
                                          page_title,
                                          RANK() OVER (PARTITION BY ga_id ORDER BY event_time ASC) AS rang,
                                          utm_campaign,
                                          utm_medium,
                                          utm_source,
                                    FROM `test29102023.logo.visits_ga`
                                    WHERE (event_name = 'first_visit')))),
                                    --AND (event_date >= '2023-06-01')) AS tab_2)

# 2. Таблица с пользователями, у которых была хотя бы одна оплата
     buyers AS (SELECT *
                FROM `test29102023.logo.visits_ga`
                WHERE ga_id IN (SELECT ga_id
                                FROM (SELECT ga_id,
                                            SUM(b_status) AS b_status
                                      FROM (SELECT CAST(ga_id AS NUMERIC) AS ga_id,
                                                        event_date,
                                                        event_time,
                                                        page_title,
                                                        CAST((CASE
                                                          WHEN page_title LIKE '%Оплата%' THEN '1'
                                                          ELSE '0'
                                                        END) AS INT) AS b_status
                                                  FROM `test29102023.logo.visits_ga`
                                          ) AS tab_1
                                      GROUP BY ga_id
                                      HAVING b_status >= 1 ) AS tab_2)),

# 3. Таблица с пользователями, у которых не было покупок
      not_buyers AS (SELECT ga_id
                     FROM `test29102023.logo.visits_ga`
                     WHERE ga_id IN (SELECT ga_id
                                     FROM (SELECT ga_id,
                                                  SUM(b_status) AS b_status
                                           FROM (SELECT CAST(ga_id AS NUMERIC) AS ga_id,
                                                        event_date,
                                                        event_time,
                                                        page_title,
                                                        CAST((CASE
                                                          WHEN page_title LIKE '%Оплата%' THEN '1'
                                                          ELSE '0'
                                                        END) AS INT) AS b_status
                                                  FROM `test29102023.logo.visits_ga`
                                          ) AS tab_1
                                      GROUP BY ga_id
                                      HAVING b_status < 1 ) AS tab_2)),

# 4. Таблица, с указанием типа продукта по каждому названию
      type_products AS (SELECT string_field_0 AS page_title,
                               string_field_1 AS type
                       FROM `test29102023.logo.type_products`)

# 5. Таблица, в которой в отдельной колонке соединены тип продукта и его номер

SELECT *,
# Преобразуем значения под требования функции построения цепей - транслит и нумерацию преобразую в буквы
       CASE
          WHEN product_way  = "Акция1" THEN "AkciaA"
          WHEN product_way  = "Акция2" THEN "AkciaB"
          WHEN product_way  = "Акция3" THEN "AkciaC"
          WHEN product_way  = "Акция4" THEN "AkciaD"
          WHEN product_way  = "Блог1" THEN "BlogA"
          WHEN product_way  = "Блог2" THEN "BlogB"
          WHEN product_way  = "Блог3" THEN "BlogC"
          WHEN product_way  = "Блог4" THEN "BlogD"
          WHEN product_way  = "Блог5" THEN "BlogE"
          WHEN product_way  = "Блог6" THEN "BlogF"
          WHEN product_way  = "Блог7" THEN "BlogG"
          WHEN product_way  = "Блог8" THEN "BlogH"
          WHEN product_way  = "Блог9" THEN "BlogI"
          WHEN product_way  = "Блог10" THEN "BlogK"
          WHEN product_way  = "Блог11" THEN "BlogL"
          WHEN product_way  = "Блог12" THEN "BlogL"
          WHEN product_way  = "Блог13" THEN "BlogL"
          WHEN product_way  = "Блог14" THEN "BlogL"
          WHEN product_way  = "Блог15" THEN "BlogL"
          WHEN product_way  = "Блог16" THEN "BlogL"
          WHEN product_way  = "Блог17" THEN "BlogL"
          WHEN product_way  = "Блог18" THEN "BlogL"
          WHEN product_way  = "Блог19" THEN "BlogL"
          WHEN product_way  = "Блог20" THEN "BlogL"
          WHEN product_way  = "Блог21" THEN "BlogL"
          WHEN product_way  = "Блог22" THEN "BlogL"
          WHEN product_way  = "Блог23" THEN "BlogL"
          WHEN product_way  = "Блог24" THEN "BlogL"
          WHEN product_way  = "Блог25" THEN "BlogL"
          WHEN product_way  = "Блог26" THEN "BlogL"
          WHEN product_way  = "Блог27" THEN "BlogL"
          WHEN product_way  = "Блог28" THEN "BlogL"
          WHEN product_way  = "Блог29" THEN "BlogL"
          WHEN product_way  = "Блог30" THEN "BlogL"
          WHEN product_way  = "Блог31" THEN "BlogL"
          WHEN product_way  = "Блог32" THEN "BlogL"
          WHEN product_way  = "Блог33" THEN "BlogL"
          WHEN product_way  = "Блог34" THEN "BlogL"
          WHEN product_way  = "Блог35" THEN "BlogL"
          WHEN product_way  = "Блог36" THEN "BlogL"
          WHEN product_way  = "Блог37" THEN "BlogL"
          WHEN product_way  = "Блог38" THEN "BlogL"
          WHEN product_way  = "Блог39" THEN "BlogL"
          WHEN product_way  = "Блог40" THEN "BlogL"
          WHEN product_way  = "Блог41" THEN "BlogL"
          WHEN product_way  = "Блог42" THEN "BlogL"
          WHEN product_way  = "Блог43" THEN "BlogL"
          WHEN product_way  = "Блог44" THEN "BlogL"
          WHEN product_way  = "Блог45" THEN "BlogL"
          WHEN product_way  = "Блог46" THEN "BlogL"
          WHEN product_way  = "Блог47" THEN "BlogL"
          WHEN product_way  = "Блог48" THEN "BlogL"
          WHEN product_way  = "Блог49" THEN "BlogL"
          WHEN product_way  = "Блог50" THEN "BlogL"
          WHEN product_way  = "Блог51" THEN "BlogL"
          WHEN product_way  = "Блог52" THEN "BlogL"
          WHEN product_way  = "Блог53" THEN "BlogL"
          WHEN product_way  = "Блог54" THEN "BlogL"
          WHEN product_way  = "Блог55" THEN "BlogL"
          WHEN product_way  = "Блог56" THEN "BlogL"
          WHEN product_way  = "Блог57" THEN "BlogL"
          WHEN product_way  = "Блог58" THEN "BlogL"
          WHEN product_way  = "Блог59" THEN "BlogL"
          WHEN product_way  = "Блог60" THEN "BlogL"
          WHEN product_way  = "Блог61" THEN "BlogL"
          WHEN product_way  = "Блог62" THEN "BlogL"
          WHEN product_way  = "Блог63" THEN "BlogL"
          WHEN product_way  = "Блог64" THEN "BlogL"
          WHEN product_way  = "Блог65" THEN "BlogL"
          WHEN product_way  = "Блог66" THEN "BlogL"
          WHEN product_way  = "Блог67" THEN "BlogL"
          WHEN product_way  = "Блог68" THEN "BlogL"
          WHEN product_way  = "Блог69" THEN "BlogL"
          WHEN product_way  = "Блог70" THEN "BlogL"
          WHEN product_way  = "Блог71" THEN "BlogL"
          WHEN product_way  = "Блог72" THEN "BlogL"
          WHEN product_way  = "Блог73" THEN "BlogL"
          WHEN product_way  = "Блог74" THEN "BlogL"
          WHEN product_way  = "Блог75" THEN "BlogL"
          WHEN product_way  = "Блог76" THEN "BlogL"
          WHEN product_way  = "Блог77" THEN "BlogL"
          WHEN product_way  = "Блог78" THEN "BlogL"
          WHEN product_way  = "Блог79" THEN "BlogL"
          WHEN product_way  = "Блог80" THEN "BlogL"
          WHEN product_way  = "Блог81" THEN "BlogL"
          WHEN product_way  = "Блог82" THEN "BlogL"
          WHEN product_way  = "Блог83" THEN "BlogL"
          WHEN product_way  = "Блог84" THEN "BlogL"
          WHEN product_way  = "Блог85" THEN "BlogL"
          WHEN product_way  = "Блог86" THEN "BlogL"
          WHEN product_way  = "Блог87" THEN "BlogL"
          WHEN product_way  = "Блог88" THEN "BlogL"
          WHEN product_way  = "Блог89" THEN "BlogL"
          WHEN product_way  = "Блог90" THEN "BlogL"
          WHEN product_way  = "Блог91" THEN "BlogL"
          WHEN product_way  = "Блог92" THEN "BlogL"
          WHEN product_way  = "Блог93" THEN "BlogL"
          WHEN product_way  = "Блог94" THEN "BlogL"
          WHEN product_way  = "Блог95" THEN "BlogL"
          WHEN product_way  = "Блог96" THEN "BlogL"
          WHEN product_way  = "Блог97" THEN "BlogL"
          WHEN product_way  = "Блог98" THEN "BlogL"
          WHEN product_way  = "Блог99" THEN "BlogL"
          WHEN product_way  = "Блог100" THEN "BlogL"
          WHEN product_way  = "Блог101" THEN "BlogL"
          WHEN product_way  = "Блог102" THEN "BlogL"
          WHEN product_way  = "Блог103" THEN "BlogL"
          WHEN product_way  = "Блог104" THEN "BlogL"
          WHEN product_way  = "Блог105" THEN "BlogL"
          WHEN product_way  = "Блог106" THEN "BlogL"
          WHEN product_way  = "Блог107" THEN "BlogL"
          WHEN product_way  = "Блог108" THEN "BlogL"
          WHEN product_way  = "Блог109" THEN "BlogL"
          WHEN product_way  = "Блог110" THEN "BlogL"
          WHEN product_way  = "Блог111" THEN "BlogL"
          WHEN product_way  = "Блог112" THEN "BlogL"
          WHEN product_way  = "Блог113" THEN "BlogL"
          WHEN product_way  = "Блог114" THEN "BlogL"
          WHEN product_way  = "Блог115" THEN "BlogL"
          WHEN product_way  = "Блог116" THEN "BlogL"
          WHEN product_way  = "Блог117" THEN "BlogL"
          WHEN product_way  = "Блог118" THEN "BlogL"
          WHEN product_way  = "Блог119" THEN "BlogL"
          WHEN product_way  = "Блог120" THEN "BlogL"
          WHEN product_way  = "Блог121" THEN "BlogL"
          WHEN product_way  = "Блог122" THEN "BlogL"
          WHEN product_way  = "Блог123" THEN "BlogL"
          WHEN product_way  = "Блог124" THEN "BlogL"
          WHEN product_way  = "Блог125" THEN "BlogL"
          WHEN product_way  = "Блог126" THEN "BlogL"
          WHEN product_way  = "Блог127" THEN "BlogL"
          WHEN product_way  = "Блог128" THEN "BlogL"
          WHEN product_way  = "Блог129" THEN "BlogL"
          WHEN product_way  = "Блог130" THEN "BlogL"
          WHEN product_way  = "Блог131" THEN "BlogL"
          WHEN product_way  = "Блог132" THEN "BlogL"
          WHEN product_way  = "Блог133" THEN "BlogL"
          WHEN product_way  = "Блог134" THEN "BlogL"
          WHEN product_way  = "Блог135" THEN "BlogL"
          WHEN product_way  = "Блог136" THEN "BlogL"
          WHEN product_way  = "Блог137" THEN "BlogL"
          WHEN product_way  = "Блог138" THEN "BlogL"
          WHEN product_way  = "Блог139" THEN "BlogL"
          WHEN product_way  = "Блог140" THEN "BlogL"
          WHEN product_way  = "Блог141" THEN "BlogL"
          WHEN product_way  = "Блог142" THEN "BlogL"
          WHEN product_way  = "Блог143" THEN "BlogL"
          WHEN product_way  = "Блог144" THEN "BlogL"
          WHEN product_way  = "Блог145" THEN "BlogL"
          WHEN product_way  = "Блог146" THEN "BlogL"
          WHEN product_way  = "Блог147" THEN "BlogL"
          WHEN product_way  = "Блог148" THEN "BlogL"
          WHEN product_way  = "Блог149" THEN "BlogL"
          WHEN product_way  = "Блог150" THEN "BlogL"
          WHEN product_way  = "Блог151" THEN "BlogL"
          WHEN product_way  = "Блог152" THEN "BlogL"
          WHEN product_way  = "Блог153" THEN "BlogL"
          WHEN product_way  = "Блог154" THEN "BlogL"
          WHEN product_way  = "Блог155" THEN "BlogL"
          WHEN product_way  = "Блог156" THEN "BlogL"
          WHEN product_way  = "Блог157" THEN "BlogL"
          WHEN product_way  = "Блог158" THEN "BlogL"
          WHEN product_way  = "Блог159" THEN "BlogL"
          WHEN product_way  = "Блог160" THEN "BlogL"
          WHEN product_way  = "Блог161" THEN "BlogL"
          WHEN product_way  = "Блог162" THEN "BlogL"
          WHEN product_way  = "Блог163" THEN "BlogL"
          WHEN product_way  = "Блог164" THEN "BlogL"
          WHEN product_way  = "Блог165" THEN "BlogL"
          WHEN product_way  = "Блог166" THEN "BlogL"
          WHEN product_way  = "Блог167" THEN "BlogL"
          WHEN product_way  = "Блог168" THEN "BlogL"
          WHEN product_way  = "Блог169" THEN "BlogL"
          WHEN product_way  = "Блог170" THEN "BlogL"
          WHEN product_way  = "Блог171" THEN "BlogL"
          WHEN product_way  = "Блог172" THEN "BlogL"
          WHEN product_way  = "Блог173" THEN "BlogL"
          WHEN product_way  = "Блог174" THEN "BlogL"
          WHEN product_way  = "Блог175" THEN "BlogL"
          WHEN product_way  = "Блог176" THEN "BlogL"
          WHEN product_way  = "Блог177" THEN "BlogL"
          WHEN product_way  = "Блог178" THEN "BlogL"
          WHEN product_way  = "Блог179" THEN "BlogL"
          WHEN product_way  = "Блог180" THEN "BlogL"
          WHEN product_way  = "Блог181" THEN "BlogL"
          WHEN product_way  = "Блог182" THEN "BlogL"
          WHEN product_way  = "Блог183" THEN "BlogL"
          WHEN product_way  = "Блог184" THEN "BlogL"
          WHEN product_way  = "Блог185" THEN "BlogL"
          WHEN product_way  = "Блог186" THEN "BlogL"
          WHEN product_way  = "Блог187" THEN "BlogL"
          WHEN product_way  = "Блог188" THEN "BlogL"
          WHEN product_way  = "Блог189" THEN "BlogL"
          WHEN product_way  = "Блог190" THEN "BlogL"
          WHEN product_way  = "Блог191" THEN "BlogL"
          WHEN product_way  = "Блог192" THEN "BlogL"
          WHEN product_way  = "Блог193" THEN "BlogL"
          WHEN product_way  = "Блог194" THEN "BlogL"
          WHEN product_way  = "Блог195" THEN "BlogL"
          WHEN product_way  = "Блог196" THEN "BlogL"
          WHEN product_way  = "Блог197" THEN "BlogL"
          WHEN product_way  = "Блог198" THEN "BlogL"
          WHEN product_way  = "Блог199" THEN "BlogL"
          WHEN product_way  = "Блог200" THEN "BlogL"
          WHEN product_way  = "Блог201" THEN "BlogL"
          WHEN product_way  = "Блог202" THEN "BlogL"
          WHEN product_way  = "Блог203" THEN "BlogL"
          WHEN product_way  = "Блог204" THEN "BlogL"
          WHEN product_way  = "Блог205" THEN "BlogL"
          WHEN product_way  = "Блог206" THEN "BlogL"
          WHEN product_way  = "Блог207" THEN "BlogL"
          WHEN product_way  = "Блог208" THEN "BlogL"
          WHEN product_way  = "Блог209" THEN "BlogL"
          WHEN product_way  = "Блог210" THEN "BlogL"
          WHEN product_way  = "Блог211" THEN "BlogL"
          WHEN product_way  = "Блог212" THEN "BlogL"
          WHEN product_way  = "Блог213" THEN "BlogL"
          WHEN product_way  = "Блог214" THEN "BlogL"
          WHEN product_way  = "Блог215" THEN "BlogL"
          WHEN product_way  = "Блог216" THEN "BlogL"
          WHEN product_way  = "Блог217" THEN "BlogL"
          WHEN product_way  = "Блог218" THEN "BlogL"
          WHEN product_way  = "Блог219" THEN "BlogL"
          WHEN product_way  = "Блог220" THEN "BlogL"
          WHEN product_way  = "Блог221" THEN "BlogL"
          WHEN product_way  = "Блог222" THEN "BlogL"
          WHEN product_way  = "Блог223" THEN "BlogL"
          WHEN product_way  = "Блог224" THEN "BlogL"
          WHEN product_way  = "Блог225" THEN "BlogL"
          WHEN product_way  = "Блог226" THEN "BlogL"
          WHEN product_way  = "Блог227" THEN "BlogL"
          WHEN product_way  = "Блог228" THEN "BlogL"
          WHEN product_way  = "Блог229" THEN "BlogL"
          WHEN product_way  = "Блог230" THEN "BlogL"
          WHEN product_way  = "Блог231" THEN "BlogL"
          WHEN product_way  = "Блог232" THEN "BlogL"
          WHEN product_way  = "Блог233" THEN "BlogL"
          WHEN product_way  = "Блог234" THEN "BlogL"
          WHEN product_way  = "Блог235" THEN "BlogL"
          WHEN product_way  = "Блог236" THEN "BlogL"
          WHEN product_way  = "Блог237" THEN "BlogL"
          WHEN product_way  = "Блог238" THEN "BlogL"
          WHEN product_way  = "Блог239" THEN "BlogL"
          WHEN product_way  = "Бонусы1" THEN "BonusA"
          WHEN product_way  = "Вебинар1" THEN "WebA"
          WHEN product_way  = "Вебинар2" THEN "WebB"
          WHEN product_way  = "Вебинар3" THEN "WebC"
          WHEN product_way  = "Вебинар4" THEN "WebD"
          WHEN product_way  = "Вебинар5" THEN "WebE"
          WHEN product_way  = "Вебинар6" THEN "WebF"
          WHEN product_way  = "Вебинар7" THEN "WebG"
          WHEN product_way  = "Вебинар8" THEN "WebH"
          WHEN product_way  = "Вебинар9" THEN "WebI"
          WHEN product_way  = "Вебинар10" THEN "WebK"
          WHEN product_way  = "Вебинар11" THEN "WebL"
          WHEN product_way  = "Вебинар12" THEN "WebL"
          WHEN product_way  = "Вебинар13" THEN "WebL"
          WHEN product_way  = "Вебинар14" THEN "WebL"
          WHEN product_way  = "Вебинар15" THEN "WebL"
          WHEN product_way  = "Вебинар16" THEN "WebL"
          WHEN product_way  = "Вебинар17" THEN "WebL"
          WHEN product_way  = "Вебинар18" THEN "WebL"
          WHEN product_way  = "Вебинар19" THEN "WebL"
          WHEN product_way  = "Вебинар20" THEN "WebL"
          WHEN product_way  = "Вебинар21" THEN "WebL"
          WHEN product_way  = "Вебинар22" THEN "WebL"
          WHEN product_way  = "Вебинар23" THEN "WebL"
          WHEN product_way  = "Вебинар24" THEN "WebL"
          WHEN product_way  = "Вебинар25" THEN "WebL"
          WHEN product_way  = "Вебинар26" THEN "WebL"
          WHEN product_way  = "Вебинар27" THEN "WebL"
          WHEN product_way  = "Вебинар28" THEN "WebL"
          WHEN product_way  = "Вебинар29" THEN "WebL"
          WHEN product_way  = "Вебинар30" THEN "WebL"
          WHEN product_way  = "Вебинар31" THEN "WebL"
          WHEN product_way  = "Вебинар32" THEN "WebL"
          WHEN product_way  = "Вебинар33" THEN "WebL"
          WHEN product_way  = "Вебинар34" THEN "WebL"
          WHEN product_way  = "Вебинар35" THEN "WebL"
          WHEN product_way  = "Вебинар36" THEN "WebL"
          WHEN product_way  = "Вебинар37" THEN "WebL"
          WHEN product_way  = "Вебинар38" THEN "WebL"
          WHEN product_way  = "Вебинар39" THEN "WebL"
          WHEN product_way  = "Вебинар40" THEN "WebL"
          WHEN product_way  = "Вебинар41" THEN "WebL"
          WHEN product_way  = "Вебинар42" THEN "WebL"
          WHEN product_way  = "Вебинар43" THEN "WebL"
          WHEN product_way  = "Вебинар44" THEN "WebL"
          WHEN product_way  = "Вебинар45" THEN "WebL"
          WHEN product_way  = "Вебинар46" THEN "WebL"
          WHEN product_way  = "Вебинар47" THEN "WebL"
          WHEN product_way  = "Вебинар48" THEN "WebL"
          WHEN product_way  = "Вебинар49" THEN "WebL"
          WHEN product_way  = "Вебинар50" THEN "WebL"
          WHEN product_way  = "Вебинар51" THEN "WebL"
          WHEN product_way  = "Вебинар52" THEN "WebL"
          WHEN product_way  = "Вебинар53" THEN "WebL"
          WHEN product_way  = "Вебинар54" THEN "WebL"
          WHEN product_way  = "Вебинар55" THEN "WebL"
          WHEN product_way  = "Вебинар56" THEN "WebL"
          WHEN product_way  = "Вебинар57" THEN "WebL"
          WHEN product_way  = "Вебинар58" THEN "WebL"
          WHEN product_way  = "Вебинар59" THEN "WebL"
          WHEN product_way  = "Вебинар60" THEN "WebL"
          WHEN product_way  = "Вебинар61" THEN "WebL"
          WHEN product_way  = "Вебинар62" THEN "WebL"
          WHEN product_way  = "Вебинар63" THEN "WebL"
          WHEN product_way  = "Вебинар64" THEN "WebL"
          WHEN product_way  = "Вебинар65" THEN "WebL"
          WHEN product_way  = "Вебинар66" THEN "WebL"
          WHEN product_way  = "Вебинар67" THEN "WebL"
          WHEN product_way  = "Вебинар68" THEN "WebL"
          WHEN product_way  = "Вебинар69" THEN "WebL"
          WHEN product_way  = "Вебинар70" THEN "WebL"
          WHEN product_way  = "Вебинар71" THEN "WebL"
          WHEN product_way  = "Вебинар72" THEN "WebL"
          WHEN product_way  = "Вебинар73" THEN "WebL"
          WHEN product_way  = "Вебинар74" THEN "WebL"
          WHEN product_way  = "Вебинар75" THEN "WebL"
          WHEN product_way  = "Вебинар76" THEN "WebL"
          WHEN product_way  = "Вебинар77" THEN "WebL"
          WHEN product_way  = "Вебинар78" THEN "WebL"
          WHEN product_way  = "Вебинар79" THEN "WebL"
          WHEN product_way  = "Вебинар80" THEN "WebL"
          WHEN product_way  = "Вебинар81" THEN "WebL"
          WHEN product_way  = "Вебинар82" THEN "WebL"
          WHEN product_way  = "Вебинар83" THEN "WebL"
          WHEN product_way  = "Вебинар84" THEN "WebL"
          WHEN product_way  = "Вебинар85" THEN "WebL"
          WHEN product_way  = "Вебинар86" THEN "WebL"
          WHEN product_way  = "Вебинар87" THEN "WebL"
          WHEN product_way  = "Вебинар88" THEN "WebL"
          WHEN product_way  = "Вебинар89" THEN "WebL"
          WHEN product_way  = "Вебинар90" THEN "WebL"
          WHEN product_way  = "Вебинар91" THEN "WebL"
          WHEN product_way  = "Вебинар92" THEN "WebL"
          WHEN product_way  = "Вебинар93" THEN "WebL"
          WHEN product_way  = "Вебинар94" THEN "WebL"
          WHEN product_way  = "Вебинар95" THEN "WebL"
          WHEN product_way  = "Вебинар96" THEN "WebL"
          WHEN product_way  = "Вебинар97" THEN "WebL"
          WHEN product_way  = "Вебинар98" THEN "WebL"
          WHEN product_way  = "Вебинар99" THEN "WebL"
          WHEN product_way  = "Вебинар100" THEN "WebL"
          WHEN product_way  = "Вебинар101" THEN "WebL"
          WHEN product_way  = "Вебинар102" THEN "WebL"
          WHEN product_way  = "Вебинар103" THEN "WebL"
          WHEN product_way  = "Вебинар104" THEN "WebL"
          WHEN product_way  = "Вебинар105" THEN "WebL"
          WHEN product_way  = "Вебинар106" THEN "WebL"
          WHEN product_way  = "Вебинар107" THEN "WebL"
          WHEN product_way  = "Вебинар108" THEN "WebL"
          WHEN product_way  = "Вебинар109" THEN "WebL"
          WHEN product_way  = "Вебинар110" THEN "WebL"
          WHEN product_way  = "Вебинар111" THEN "WebL"
          WHEN product_way  = "Вебинар112" THEN "WebL"
          WHEN product_way  = "Вебинар113" THEN "WebL"
          WHEN product_way  = "Вебинар114" THEN "WebL"
          WHEN product_way  = "Вебинар115" THEN "WebL"
          WHEN product_way  = "Вебинар116" THEN "WebL"
          WHEN product_way  = "Вебинар117" THEN "WebL"
          WHEN product_way  = "Вебинар118" THEN "WebL"
          WHEN product_way  = "Вебинар119" THEN "WebL"
          WHEN product_way  = "Вебинар120" THEN "WebL"
          WHEN product_way  = "Вебинар121" THEN "WebL"
          WHEN product_way  = "Вебинар122" THEN "WebL"
          WHEN product_way  = "Вебинар123" THEN "WebL"
          WHEN product_way  = "Вебинар124" THEN "WebL"
          WHEN product_way  = "Вебинар125" THEN "WebL"
          WHEN product_way  = "Вебинар126" THEN "WebL"
          WHEN product_way  = "Вебинар127" THEN "WebL"
          WHEN product_way  = "Вебинар128" THEN "WebL"
          WHEN product_way  = "Вебинар129" THEN "WebL"
          WHEN product_way  = "Вебинар130" THEN "WebL"
          WHEN product_way  = "Вебинар131" THEN "WebL"
          WHEN product_way  = "Вебинар132" THEN "WebL"
          WHEN product_way  = "Вебинар133" THEN "WebL"
          WHEN product_way  = "Вебинар134" THEN "WebL"
          WHEN product_way  = "Вебинар135" THEN "WebL"
          WHEN product_way  = "Вебинар136" THEN "WebL"
          WHEN product_way  = "Вебинар137" THEN "WebL"
          WHEN product_way  = "Вебинар138" THEN "WebL"
          WHEN product_way  = "Вебинар139" THEN "WebL"
          WHEN product_way  = "Вебинар140" THEN "WebL"
          WHEN product_way  = "Вебинар141" THEN "WebL"
          WHEN product_way  = "Вебинар142" THEN "WebL"
          WHEN product_way  = "Вебинар143" THEN "WebL"
          WHEN product_way  = "Вебинар144" THEN "WebL"
          WHEN product_way  = "Вебинар145" THEN "WebL"
          WHEN product_way  = "Вебинар146" THEN "WebL"
          WHEN product_way  = "Вебинар147" THEN "WebL"
          WHEN product_way  = "Вебинар148" THEN "WebL"
          WHEN product_way  = "Вебинар149" THEN "WebL"
          WHEN product_way  = "Вебинар150" THEN "WebL"
          WHEN product_way  = "Вебинар151" THEN "WebL"
          WHEN product_way  = "Вебинар152" THEN "WebL"
          WHEN product_way  = "Вебинар153" THEN "WebL"
          WHEN product_way  = "Вебинар_апселл1" THEN "WebL"
          WHEN product_way  = "Вебинар_апселл2" THEN "WebL"
          WHEN product_way  = "Вебинары1" THEN "WebA"
          WHEN product_way  = "Главная1" THEN "MainA"
          WHEN product_way  = "Главная2" THEN "MainB"
          WHEN product_way  = "Интенсив1" THEN "IntA"
          WHEN product_way  = "Интенсив2" THEN "IntB"
          WHEN product_way  = "Интенсив3" THEN "IntC"
          WHEN product_way  = "Интенсив4" THEN "IntD"
          WHEN product_way  = "Интенсив5" THEN "IntE"
          WHEN product_way  = "Интенсив6" THEN "IntF"
          WHEN product_way  = "Интенсив7" THEN "IntG"
          WHEN product_way  = "Интенсив8" THEN "IntH"
          WHEN product_way  = "Интенсив9" THEN "IntI"
          WHEN product_way  = "Интенсив10" THEN "IntK"
          WHEN product_way  = "Интенсив11" THEN "IntL"
          WHEN product_way  = "Интенсив12" THEN "IntL"
          WHEN product_way  = "Интенсив13" THEN "IntL"
          WHEN product_way  = "Интенсив14" THEN "IntL"
          WHEN product_way  = "Интенсив15" THEN "IntL"
          WHEN product_way  = "Интенсив16" THEN "IntL"
          WHEN product_way  = "Интенсив17" THEN "IntL"
          WHEN product_way  = "Интенсив18" THEN "IntL"
          WHEN product_way  = "Интенсив19" THEN "IntL"
          WHEN product_way  = "Интенсив20" THEN "IntL"
          WHEN product_way  = "Интенсив21" THEN "IntL"
          WHEN product_way  = "Интенсив22" THEN "IntL"
          WHEN product_way  = "Интенсив23" THEN "IntL"
          WHEN product_way  = "Интенсив24" THEN "IntL"
          WHEN product_way  = "Интенсив25" THEN "IntL"
          WHEN product_way  = "Интенсив26" THEN "IntL"
          WHEN product_way  = "Интенсив27" THEN "IntL"
          WHEN product_way  = "Интенсив28" THEN "IntL"
          WHEN product_way  = "Интенсив29" THEN "IntL"
          WHEN product_way  = "Интенсив30" THEN "IntL"
          WHEN product_way  = "Интенсив31" THEN "IntL"
          WHEN product_way  = "Интенсив32" THEN "IntL"
          WHEN product_way  = "Интенсив33" THEN "IntL"
          WHEN product_way  = "Интенсив34" THEN "IntL"
          WHEN product_way  = "Интенсив35" THEN "IntL"
          WHEN product_way  = "Интенсив36" THEN "IntL"
          WHEN product_way  = "Интенсив37" THEN "IntL"
          WHEN product_way  = "Интенсив38" THEN "IntL"
          WHEN product_way  = "Интенсив39" THEN "IntL"
          WHEN product_way  = "Интенсив40" THEN "IntL"
          WHEN product_way  = "Интенсив41" THEN "IntL"
          WHEN product_way  = "Интенсив42" THEN "IntL"
          WHEN product_way  = "Интенсив43" THEN "IntL"
          WHEN product_way  = "Интенсив44" THEN "IntL"
          WHEN product_way  = "Интенсив45" THEN "IntL"
          WHEN product_way  = "Интенсив46" THEN "IntL"
          WHEN product_way  = "Интенсив47" THEN "IntL"
          WHEN product_way  = "Интенсив48" THEN "IntL"
          WHEN product_way  = "Интенсив49" THEN "IntL"
          WHEN product_way  = "Интенсив50" THEN "IntL"
          WHEN product_way  = "Интенсив51" THEN "IntL"
          WHEN product_way  = "Интенсив52" THEN "IntL"
          WHEN product_way  = "Интенсив53" THEN "IntL"
          WHEN product_way  = "Интенсив54" THEN "IntL"
          WHEN product_way  = "Интенсив55" THEN "IntL"
          WHEN product_way  = "Интенсив56" THEN "IntL"
          WHEN product_way  = "Интенсив57" THEN "IntL"
          WHEN product_way  = "Интенсив58" THEN "IntL"
          WHEN product_way  = "Интенсив59" THEN "IntL"
          WHEN product_way  = "Интенсив60" THEN "IntL"
          WHEN product_way  = "Интенсив61" THEN "IntL"
          WHEN product_way  = "Интенсив62" THEN "IntL"
          WHEN product_way  = "Интенсив63" THEN "IntL"
          WHEN product_way  = "Интенсив64" THEN "IntL"
          WHEN product_way  = "Интенсив65" THEN "IntL"
          WHEN product_way  = "Интенсив66" THEN "IntL"
          WHEN product_way  = "Интенсив67" THEN "IntL"
          WHEN product_way  = "Интенсив68" THEN "IntL"
          WHEN product_way  = "Интенсив69" THEN "IntL"
          WHEN product_way  = "Интенсив70" THEN "IntL"
          WHEN product_way  = "Интенсив71" THEN "IntL"
          WHEN product_way  = "Интенсив72" THEN "IntL"
          WHEN product_way  = "Интенсив73" THEN "IntL"
          WHEN product_way  = "Интенсив74" THEN "IntL"
          WHEN product_way  = "Интенсив75" THEN "IntL"
          WHEN product_way  = "Интенсив76" THEN "IntL"
          WHEN product_way  = "Интенсив77" THEN "IntL"
          WHEN product_way  = "Интенсив78" THEN "IntL"
          WHEN product_way  = "Конференция1" THEN "ConfA"
          WHEN product_way  = "Курс1" THEN "KursA"
          WHEN product_way  = "Курс2" THEN "KursB"
          WHEN product_way  = "Курс3" THEN "KursC"
          WHEN product_way  = "Курс4" THEN "KursD"
          WHEN product_way  = "Курс5" THEN "KursE"
          WHEN product_way  = "Курс6" THEN "KursF"
          WHEN product_way  = "Курс7" THEN "KursG"
          WHEN product_way  = "Курс8" THEN "KursH"
          WHEN product_way  = "Курс9" THEN "KursI"
          WHEN product_way  = "Курс10" THEN "KursK"
          WHEN product_way  = "Курс11" THEN "KursL"
          WHEN product_way  = "Курс12" THEN "KursL"
          WHEN product_way  = "Курс13" THEN "KursL"
          WHEN product_way  = "Курс14" THEN "KursL"
          WHEN product_way  = "Курс15" THEN "KursL"
          WHEN product_way  = "Курс16" THEN "KursL"
          WHEN product_way  = "Курс17" THEN "KursL"
          WHEN product_way  = "Курс18" THEN "KursL"
          WHEN product_way  = "Курс19" THEN "KursL"
          WHEN product_way  = "Курс20" THEN "KursL"
          WHEN product_way  = "Курс21" THEN "KursL"
          WHEN product_way  = "Курс22" THEN "KursL"
          WHEN product_way  = "Курс23" THEN "KursL"
          WHEN product_way  = "Курс24" THEN "KursL"
          WHEN product_way  = "Курс25" THEN "KursL"
          WHEN product_way  = "Курс26" THEN "KursL"
          WHEN product_way  = "Курс27" THEN "KursL"
          WHEN product_way  = "Курс28" THEN "KursL"
          WHEN product_way  = "Курс29" THEN "KursL"
          WHEN product_way  = "Курс30" THEN "KursL"
          WHEN product_way  = "Курс31" THEN "KursL"
          WHEN product_way  = "Курс32" THEN "KursL"
          WHEN product_way  = "Курс33" THEN "KursL"
          WHEN product_way  = "Курс34" THEN "KursL"
          WHEN product_way  = "Курс35" THEN "KursL"
          WHEN product_way  = "Курс36" THEN "KursL"
          WHEN product_way  = "Курс37" THEN "KursL"
          WHEN product_way  = "Курс38" THEN "KursL"
          WHEN product_way  = "Курс39" THEN "KursL"
          WHEN product_way  = "Курс40" THEN "KursL"
          WHEN product_way  = "Курс41" THEN "KursL"
          WHEN product_way  = "Курс42" THEN "KursL"
          WHEN product_way  = "Курс43" THEN "KursL"
          WHEN product_way  = "Курс44" THEN "KursL"
          WHEN product_way  = "Курс45" THEN "KursL"
          WHEN product_way  = "Курс46" THEN "KursL"
          WHEN product_way  = "Курс47" THEN "KursL"
          WHEN product_way  = "Курс48" THEN "KursL"
          WHEN product_way  = "Курс49" THEN "KursL"
          WHEN product_way  = "Курс50" THEN "KursL"
          WHEN product_way  = "Курс51" THEN "KursL"
          WHEN product_way  = "Курс52" THEN "KursL"
          WHEN product_way  = "Лид-магнит1" THEN "LmA"
          WHEN product_way  = "Лид-магнит2" THEN "LmB"
          WHEN product_way  = "Лид-магнит3" THEN "LmC"
          WHEN product_way  = "Лид-магнит4" THEN "LmD"
          WHEN product_way  = "Лид-магнит5" THEN "LmE"
          WHEN product_way  = "Лид-магнит6" THEN "LmF"
          WHEN product_way  = "Лид-магнит7" THEN "LmG"
          WHEN product_way  = "Лид-магнит8" THEN "LmH"
          WHEN product_way  = "Лид-магнит9" THEN "LmI"
          WHEN product_way  = "Лид-магнит10" THEN "LmK"
          WHEN product_way  = "Лид-магнит11" THEN "LmL"
          WHEN product_way  = "Лид-магнит12" THEN "LmL"
          WHEN product_way  = "Лид-магнит13" THEN "LmL"
          WHEN product_way  = "Лид-магнит14" THEN "LmL"
          WHEN product_way  = "Лид-магнит15" THEN "LmL"
          WHEN product_way  = "Лид-магнит16" THEN "LmL"
          WHEN product_way  = "Лид-магнит17" THEN "LmL"
          WHEN product_way  = "Лид-магнит18" THEN "LmL"
          WHEN product_way  = "Лид-магнит19" THEN "LmL"
          WHEN product_way  = "Лид-магнит20" THEN "LmL"
          WHEN product_way  = "Лид-магнит21" THEN "LmL"
          WHEN product_way  = "Лид-магнит22" THEN "LmL"
          WHEN product_way  = "Лид-магнит23" THEN "LmL"
          WHEN product_way  = "Лид-магнит24" THEN "LmL"
          WHEN product_way  = "Лид-магнит25" THEN "LmL"
          WHEN product_way  = "Лид-магнит26" THEN "LmL"
          WHEN product_way  = "Лид-магнит27" THEN "LmL"
          WHEN product_way  = "Лид-магнит28" THEN "LmL"
          WHEN product_way  = "Лид-магнит29" THEN "LmL"
          WHEN product_way  = "Лид-магнит30" THEN "LmL"
          WHEN product_way  = "Лид-магнит31" THEN "LmL"
          WHEN product_way  = "Лид-магнит32" THEN "LmL"
          WHEN product_way  = "Лид-магнит33" THEN "LmL"
          WHEN product_way  = "Лид-магнит34" THEN "LmL"
          WHEN product_way  = "Лид-магнит35" THEN "LmL"
          WHEN product_way  = "Лид-магнит36" THEN "LmL"
          WHEN product_way  = "Лид-магнит37" THEN "LmL"
          WHEN product_way  = "Лид-магнит38" THEN "LmL"
          WHEN product_way  = "Лид-магнит39" THEN "LmL"
          WHEN product_way  = "Лид-магнит40" THEN "LmL"
          WHEN product_way  = "Лид-магнит41" THEN "LmL"
          WHEN product_way  = "Логошоу1" THEN "ShowA"
          WHEN product_way  = "Логошоу2" THEN "ShowB"
          WHEN product_way  = "Логошоу3" THEN "ShowC"
          WHEN product_way  = "Логошоу4" THEN "ShowD"
          WHEN product_way  = "Мастер-класс1" THEN "MkA"
          WHEN product_way  = "Мастер-класс2" THEN "MkB"
          WHEN product_way  = "Мастер-класс3" THEN "MkC"
          WHEN product_way  = "Мастер-класс4" THEN "MkD"
          WHEN product_way  = "Мастер-класс5" THEN "MkE"
          WHEN product_way  = "Мастер-класс6" THEN "MkF"
          WHEN product_way  = "Мастер-класс7" THEN "MkG"
          WHEN product_way  = "Мастер-класс8" THEN "MkH"
          WHEN product_way  = "Мастер-класс9" THEN "MkI"
          WHEN product_way  = "Мастер-класс10" THEN "MkK"
          WHEN product_way  = "Мастер-класс11" THEN "MkL"
          WHEN product_way  = "Мастер-класс12" THEN "MkL"
          WHEN product_way  = "Мастер-класс13" THEN "MkL"
          WHEN product_way  = "Мастер-класс14" THEN "MkL"
          WHEN product_way  = "Мастер-класс15" THEN "MkL"
          WHEN product_way  = "Мастер-класс16" THEN "MkL"
          WHEN product_way  = "Мастер-класс17" THEN "MkL"
          WHEN product_way  = "Мастер-класс18" THEN "MkL"
          WHEN product_way  = "Мастер-класс19" THEN "MkL"
          WHEN product_way  = "Мастер-класс20" THEN "MkL"
          WHEN product_way  = "Мастер-класс21" THEN "MkL"
          WHEN product_way  = "Мастер-класс22" THEN "MkL"
          WHEN product_way  = "Мастер-класс23" THEN "MkL"
          WHEN product_way  = "Мастер-класс24" THEN "MkL"
          WHEN product_way  = "Мастер-класс25" THEN "MkL"
          WHEN product_way  = "Мастер-класс26" THEN "MkL"
          WHEN product_way  = "Мастер-класс27" THEN "MkL"
          WHEN product_way  = "Мастер-класс28" THEN "MkL"
          WHEN product_way  = "Мастер-класс29" THEN "MkL"
          WHEN product_way  = "Мастер-класс30" THEN "MkL"
          WHEN product_way  = "Мастер-класс31" THEN "MkL"
          WHEN product_way  = "Мастер-класс32" THEN "MkL"
          WHEN product_way  = "Мастер-класс33" THEN "MkL"
          WHEN product_way  = "Мастер-класс34" THEN "MkL"
          WHEN product_way  = "Мастер-класс35" THEN "MkL"
          WHEN product_way  = "Мастер-класс36" THEN "MkL"
          WHEN product_way  = "Мастер-класс37" THEN "MkL"
          WHEN product_way  = "Мастер-класс38" THEN "MkL"
          WHEN product_way  = "Мастер-класс39" THEN "MkL"
          WHEN product_way  = "Мастер-класс40" THEN "MkL"
          WHEN product_way  = "Мастер-класс41" THEN "MkL"
          WHEN product_way  = "Мастер-класс42" THEN "MkL"
          WHEN product_way  = "Мастер-класс43" THEN "MkL"
          WHEN product_way  = "Мастер-класс44" THEN "MkL"
          WHEN product_way  = "Мастер-класс45" THEN "MkL"
          WHEN product_way  = "Мастер-класс46" THEN "MkL"
          WHEN product_way  = "Мастер-класс47" THEN "MkL"
          WHEN product_way  = "Мастер-класс48" THEN "MkL"
          WHEN product_way  = "Мастер-класс49" THEN "MkL"
          WHEN product_way  = "Мастер-класс50" THEN "MkL"
          WHEN product_way  = "Мастер-класс51" THEN "MkL"
          WHEN product_way  = "Мастер-класс52" THEN "MkL"
          WHEN product_way  = "Мастер-класс53" THEN "MkL"
          WHEN product_way  = "Мастер-класс54" THEN "MkL"
          WHEN product_way  = "Мастер-класс55" THEN "MkL"
          WHEN product_way  = "Мастер-класс56" THEN "MkL"
          WHEN product_way  = "Мастер-класс57" THEN "MkL"
          WHEN product_way  = "Мастер-класс58" THEN "MkL"
          WHEN product_way  = "Мастер-класс59" THEN "MkL"
          WHEN product_way  = "Мастер-класс60" THEN "MkL"
          WHEN product_way  = "Мастер-класс61" THEN "MkL"
          WHEN product_way  = "Мастер-класс62" THEN "MkL"
          WHEN product_way  = "Мастер-класс63" THEN "MkL"
          WHEN product_way  = "Мастер-класс64" THEN "MkL"
          WHEN product_way  = "Мастер-класс65" THEN "MkL"
          WHEN product_way  = "Мастер-класс66" THEN "MkL"
          WHEN product_way  = "Мастер-класс67" THEN "MkL"
          WHEN product_way  = "Мастер-класс68" THEN "MkL"
          WHEN product_way  = "Мастер-класс69" THEN "MkL"
          WHEN product_way  = "Мастер-класс70" THEN "MkL"
          WHEN product_way  = "Мастер-класс71" THEN "MkL"
          WHEN product_way  = "Мастер-класс72" THEN "MkL"
          WHEN product_way  = "Мастер-класс73" THEN "MkL"
          WHEN product_way  = "Мастер-класс74" THEN "MkL"
          WHEN product_way  = "Мастер-класс75" THEN "MkL"
          WHEN product_way  = "Мастер-класс76" THEN "MkL"
          WHEN product_way  = "Мастер-класс77" THEN "MkL"
          WHEN product_way  = "Мастер-класс78" THEN "MkL"
          WHEN product_way  = "Мастер-класс79" THEN "MkL"
          WHEN product_way  = "Мастер-класс80" THEN "MkL"
          WHEN product_way  = "Мастер-класс81" THEN "MkL"
          WHEN product_way  = "Мастер-класс82" THEN "MkL"
          WHEN product_way  = "Мастер-класс83" THEN "MkL"
          WHEN product_way  = "Мастер-класс84" THEN "MkL"
          WHEN product_way  = "Мастер-класс85" THEN "MkL"
          WHEN product_way  = "Мастер-класс86" THEN "MkL"
          WHEN product_way  = "Мастер-класс87" THEN "MkL"
          WHEN product_way  = "Мастер-класс88" THEN "MkL"
          WHEN product_way  = "Мастер-класс89" THEN "MkL"
          WHEN product_way  = "Мастер-класс90" THEN "MkL"
          WHEN product_way  = "Мастер-класс91" THEN "MkL"
          WHEN product_way  = "Мастер-класс92" THEN "MkL"
          WHEN product_way  = "Мастер-класс93" THEN "MkL"
          WHEN product_way  = "Мастер-класс94" THEN "MkL"
          WHEN product_way  = "Мастер-класс95" THEN "MkL"
          WHEN product_way  = "Мастер-класс96" THEN "MkL"
          WHEN product_way  = "Мастер-класс97" THEN "MkL"
          WHEN product_way  = "Мастер-класс98" THEN "MkL"
          WHEN product_way  = "Мастер-класс99" THEN "MkL"
          WHEN product_way  = "Мастер-класс100" THEN "MkL"
          WHEN product_way  = "Мастер-класс101" THEN "MkL"
          WHEN product_way  = "Мастер-класс102" THEN "MkL"
          WHEN product_way  = "Мастер-класс103" THEN "MkL"
          WHEN product_way  = "Мастер-класс104" THEN "MkL"
          WHEN product_way  = "Мастер-класс105" THEN "MkL"
          WHEN product_way  = "Мастер-класс106" THEN "MkL"
          WHEN product_way  = "Мастер-класс107" THEN "MkL"
          WHEN product_way  = "Мастер-класс108" THEN "MkL"
          WHEN product_way  = "Мастер-класс109" THEN "MkL"
          WHEN product_way  = "Мастер-класс110" THEN "MkL"
          WHEN product_way  = "Мастер-класс111" THEN "MkL"
          WHEN product_way  = "Мастер-класс112" THEN "MkL"
          WHEN product_way  = "Мастер-класс113" THEN "MkL"
          WHEN product_way  = "Мастер-класс114" THEN "MkL"
          WHEN product_way  = "Мастер-класс115" THEN "MkL"
          WHEN product_way  = "Нескучная логопедия1" THEN "FunA"
          WHEN product_way  = "Нескучная логопедия2" THEN "FunB"
          WHEN product_way  = "Нескучная логопедия3" THEN "FunC"
          WHEN product_way  = "Нескучная логопедия4" THEN "FunD"
          WHEN product_way  = "Нескучная логопедия5" THEN "FunE"
          WHEN product_way  = "Нескучная логопедия6" THEN "FunF"
          WHEN product_way  = "Нескучная логопедия7" THEN "FunG"
          WHEN product_way  = "Нескучная логопедия8" THEN "FunH"
          WHEN product_way  = "Нескучная логопедия9" THEN "FunI"
          WHEN product_way  = "Нескучная логопедия10" THEN "FunK"
          WHEN product_way  = "Нескучная логопедия11" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия12" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия13" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия14" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия15" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия16" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия17" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия18" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия19" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия20" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия21" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия22" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия23" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия24" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия25" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия26" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия27" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия28" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия29" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия30" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия31" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия32" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия33" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия34" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия35" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия36" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия37" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия38" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия39" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия40" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия41" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия42" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия43" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия44" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия45" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия46" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия47" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия48" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия49" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия50" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия51" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия52" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия53" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия54" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия55" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия56" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия57" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия58" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия59" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия60" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия61" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия62" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия63" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия64" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия65" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия66" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия67" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия68" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия69" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия70" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия71" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия72" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия73" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия74" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия75" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия76" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия77" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия78" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия79" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия80" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия81" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия82" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия83" THEN "FunL"
          WHEN product_way  = "Нескучная логопедия84" THEN "FunL"
          WHEN product_way  = "Подписка1" THEN "SubsA"
          WHEN product_way  = "Подписка2" THEN "SubsB"
          WHEN product_way  = "Подписка3" THEN "SubsC"
          WHEN product_way  = "Подписка4" THEN "SubsD"
          WHEN product_way  = "Подписка5" THEN "SubsE"
          WHEN product_way  = "Подписка6" THEN "SubsF"
          WHEN product_way  = "Подписка7" THEN "SubsG"
          WHEN product_way  = "Подписка8" THEN "SubsH"
          WHEN product_way  = "Подписка9" THEN "SubsI"
          WHEN product_way  = "Подписка10" THEN "SubsK"
          WHEN product_way  = "Подписка11" THEN "SubsL"
          WHEN product_way  = "Подписка12" THEN "SubsL"
          WHEN product_way  = "Подписка13" THEN "SubsL"
          WHEN product_way  = "Подписка14" THEN "SubsL"
          WHEN product_way  = "Подписка15" THEN "SubsL"
          WHEN product_way  = "Подписка16" THEN "SubsL"
          WHEN product_way  = "Подписка17" THEN "SubsL"
          WHEN product_way  = "Подписка18" THEN "SubsL"
          WHEN product_way  = "Подписка19" THEN "SubsL"
          WHEN product_way  = "Подписка20" THEN "SubsL"
          WHEN product_way  = "Подписка21" THEN "SubsL"
          WHEN product_way  = "Подписка22" THEN "SubsL"
          WHEN product_way  = "Подписка23" THEN "SubsL"
          WHEN product_way  = "Подписка24" THEN "SubsL"
          WHEN product_way  = "Подписка25" THEN "SubsL"
          WHEN product_way  = "Подписка26" THEN "SubsL"
          WHEN product_way  = "Подписка27" THEN "SubsL"
          WHEN product_way  = "Подписка28" THEN "SubsL"
          WHEN product_way  = "Подписка29" THEN "SubsL"
          WHEN product_way  = "Подписка30" THEN "SubsL"
          WHEN product_way  = "Подписка31" THEN "SubsL"
          WHEN product_way  = "Подписка32" THEN "SubsL"
          WHEN product_way  = "Подписка33" THEN "SubsL"
          WHEN product_way  = "Подписка34" THEN "SubsL"
          WHEN product_way  = "Подписка35" THEN "SubsL"
          WHEN product_way  = "Подписка36" THEN "SubsL"
          WHEN product_way  = "Подписка37" THEN "SubsL"
          WHEN product_way  = "Подписка38" THEN "SubsL"
          WHEN product_way  = "Подписка39" THEN "SubsL"
          WHEN product_way  = "Подписка40" THEN "SubsL"
          WHEN product_way  = "Подписка41" THEN "SubsL"
          WHEN product_way  = "Подписка42" THEN "SubsL"
          WHEN product_way  = "Рабочая программа1" THEN "WorkA"
          WHEN product_way  = "Рабочая программа2" THEN "WorkB"
          WHEN product_way  = "Рабочая программа3" THEN "WorkC"
          WHEN product_way  = "Рабочая программа4" THEN "WorkD"
          WHEN product_way  = "Супервизия1" THEN "SuperA"
          WHEN product_way  = "Супервизия2" THEN "SuperB"
          WHEN product_way  = "Трипвайер1" THEN "TripA"
          WHEN product_way  = "Трипвайер2" THEN "TripB"
          WHEN product_way  = "Трипвайер3" THEN "TripC"
          WHEN product_way  = "Трипвайер4" THEN "TripD"
          WHEN product_way  = "Трипвайер5" THEN "TripE"
          WHEN product_way  = "Трипвайер6" THEN "TripF"
          WHEN product_way  = "Трипвайер7" THEN "TripG"
          WHEN product_way  = "Трипвайер8" THEN "TripH"
          WHEN product_way  = "Трипвайер9" THEN "TripI"
          WHEN product_way  = "Трипвайер10" THEN "TripK"
          WHEN product_way  = "Трипвайер11" THEN "TripL"
          WHEN product_way  = "Трипвайер12" THEN "TripL"
          WHEN product_way  = "Трипвайер13" THEN "TripL"
          WHEN product_way  = "Трипвайер14" THEN "TripL"
          WHEN product_way  = "Трипвайер15" THEN "TripL"
          WHEN product_way  = "Трипвайер16" THEN "TripL"
          WHEN product_way  = "Трипвайер17" THEN "TripL"
          WHEN product_way  = "Трипвайер18" THEN "TripL"
          WHEN product_way  = "Трипвайер19" THEN "TripL"
          WHEN product_way  = "Трипвайер20" THEN "TripL"
          WHEN product_way  = "Трипвайер21" THEN "TripL"
          WHEN product_way  = "Трипвайер22" THEN "TripL"
          WHEN product_way  = "Трипвайер23" THEN "TripL"
          WHEN product_way  = "Трипвайер24" THEN "TripL"
          WHEN product_way  = "Трипвайер25" THEN "TripL"
          WHEN product_way  = "Трипвайер26" THEN "TripL"
          WHEN product_way  = "Трипвайер27" THEN "TripL"
          WHEN product_way  = "Трипвайер28" THEN "TripL"
          WHEN product_way  = "Трипвайер29" THEN "TripL"
          WHEN product_way  = "Трипвайер30" THEN "TripL"
          WHEN product_way  = "Трипвайер31" THEN "TripL"
          WHEN product_way  = "Трипвайер32" THEN "TripL"
          WHEN product_way  = "Трипвайер33" THEN "TripL"
          WHEN product_way  = "Трипвайер34" THEN "TripL"
          WHEN product_way  = "Трипвайер35" THEN "TripL"
          WHEN product_way  = "Трипвайер36" THEN "TripL"
          WHEN product_way  = "Трипвайер37" THEN "TripL"
          WHEN product_way  = "Трипвайер38" THEN "TripL"
          WHEN product_way  = "Трипвайер39" THEN "TripL"
          WHEN product_way  = "Трипвайер40" THEN "TripL"
          WHEN product_way  = "Трипвайер41" THEN "TripL"
          WHEN product_way  = "Трипвайер42" THEN "TripL"
          WHEN product_way  = "Трипвайер43" THEN "TripL"
          WHEN product_way  = "Трипвайер44" THEN "TripL"
          WHEN product_way  = "Трипвайер45" THEN "TripL"
          WHEN product_way  = "Трипвайер46" THEN "TripL"
          WHEN product_way  = "Трипвайер47" THEN "TripL"
          WHEN product_way  = "Трипвайер48" THEN "TripL"
          WHEN product_way  = "Трипвайер49" THEN "TripL"
          WHEN product_way  = "Трипвайер50" THEN "TripL"
          WHEN product_way  = "Трипвайер51" THEN "TripL"
          WHEN product_way  = "Трипвайер52" THEN "TripL"
          WHEN product_way  = "Трипвайер53" THEN "TripL"
          WHEN product_way  = "Трипвайер54" THEN "TripL"
          WHEN product_way  = "Трипвайер55" THEN "TripL"
          WHEN product_way  = "Трипвайер56" THEN "TripL"
          WHEN product_way  = "Трипвайер57" THEN "TripL"
          WHEN product_way  = "Трипвайер58" THEN "TripL"
          WHEN product_way  = "Трипвайер59" THEN "TripL"
          WHEN product_way  = "Трипвайер60" THEN "TripL"
          WHEN product_way  = "Трипвайер61" THEN "TripL"
          WHEN product_way  = "Трипвайер62" THEN "TripL"
          WHEN product_way  = "Трипвайер63" THEN "TripL"
          WHEN product_way  = "Трипвайер64" THEN "TripL"
          WHEN product_way  = "Трипвайер65" THEN "TripL"
          WHEN product_way  = "Трипвайер66" THEN "TripL"
          WHEN product_way  = "Трипвайер67" THEN "TripL"
          WHEN product_way  = "Трипвайер68" THEN "TripL"
       END AS product_type
FROM (SELECT *,
            CONCAT(type, '', rn) AS product_way
      FROM (
      SELECT tab_1.ga_id AS ga_id,
            tab_1.event_date AS event_date,
            tab_1.event_name AS event_name,
            tab_1.event_time AS event_time,
            tab_1.page_title AS page_title,
            tab_1.rang AS rang ,
            tab_1.utm_campaign AS utm_campaign ,
            tab_1.utm_medium AS utm_medium ,
            tab_1.utm_source AS utm_source ,
            tab_3.type AS type ,
            ROW_NUMBER() over (partition by tab_1.ga_id, type order by tab_1.event_time) as rn
      FROM (# 5.1 Таблица, в которой проранжированы посещения страниц в рамках каждого типа продукта
            SELECT *
            FROM users
            WHERE ga_id IN (SELECT ga_id
                          FROM `test29102023.logo.visits_ga`
                          WHERE ga_id IN (SELECT ga_id
                                          FROM (SELECT ga_id,
                                                        SUM(b_status) AS b_status
                                                FROM (SELECT CAST(ga_id AS NUMERIC) AS ga_id,
                                                              event_date,
                                                              event_time,
                                                              page_title,
                                                              CAST((CASE
                                                                WHEN page_title LIKE '%Оплата%' THEN '1'
                                                                ELSE '0'
                                                              END) AS INT) AS b_status
                                                        FROM `test29102023.logo.visits_ga`
                                                ) AS tab_1
                                            GROUP BY ga_id
                                            HAVING b_status < 1 ) AS tab_2))) AS tab_1
      LEFT JOIN type_products AS tab_3 ON tab_3.page_title = tab_1.page_title
      WHERE (tab_1.page_title != 'None')
      AND (tab_3.type != 'Техническое')
      AND (tab_3.type != 'Тестирование')
      AND (tab_3.type != 'Удостоверение')
      AND (tab_3.type != 'Заявки на вебинар')
      AND (tab_3.type != 'Заявка')
      AND (tab_3.type != 'Заказ')
      AND (tab_1.page_title != 'None')
      AND (tab_3.type != 'Покупка')
      AND (tab_3.type != 'Лекция')
      ) AS main_tab ) AS tab
''')

In [103]:
buyers_no_chain = client.query(chain_no_conv).to_dataframe()
buyers_no_chain.head(5)

,ga_id,event_date,event_name,event_time,page_title,rang,utm_campaign,utm_medium,utm_source,type,rn,product_way,product_type
0,32725729.168023606,2023-06-05,page_view,2023-06-05 17:20:49+00:00,Логотека,39,reklama,podslysheno,vk,Курс,2,Курс2,KursB
1,32725729.168023606,2023-06-05,page_view,2023-06-05 17:21:52+00:00,Практические материалы для работы логопедов и коррекционных специалистов,40,reklama,podslysheno,vk,Курс,3,Курс3,KursC
2,40474410.167781444,2023-03-03,page_view,2023-03-03 04:31:48+00:00,25-12-2019 Учимся играя,8,(referral),referral,yandex.ru,Вебинар,2,Вебинар2,WebB
3,41947628.167842890,2023-03-10,page_view,2023-03-10 13:00:27+00:00,"Дракон, Паук и Осьминог – Логопедический игротренинг. Блог Лого-Эксперт",10,(direct),(none),(direct),Блог,9,Блог9,BlogI
4,41947628.167842890,2023-03-10,page_view,2023-03-10 13:06:06+00:00,Картотека дидактических игр. Блог Лого-Эксперт,11,(direct),(none),(direct),Блог,10,Блог10,BlogK


### `21. Сформированные датафреймы на основе запросов п.20`<a id="step21"></a>

 [Вернуться в начало](#stepend)</n>

21.1 Датафрейм с данными покупателей<a id="step21_1"></a>

 [Вернуться в начало](#stepend)</n>

In [104]:
buyers_chain.head(5)

,ga_id,event_date,event_time,event_name,page_title,rang,type,rn,product_way,product_type
0,2136012.168544892,2023-05-30,2023-05-30 12:15:22+00:00,first_visit,«Структура и содержание индивидуального логопедического занятия в ДОО»,1,Интенсив,1,Интенсив1,IntA
1,14268773.160667584,2023-04-16,2023-04-16 05:03:08+00:00,first_visit,Чемоданчик логопеда | Дошкольная Академия,1,Мастер-класс,1,Мастер-класс1,MkA
2,26565126.167655439,2023-02-16,2023-02-16 13:33:12+00:00,first_visit,Подборка коллажей с 1 по 8 тип слоговой структуры слова. Блог Лого-Эксперт,1,Блог,1,Блог1,BlogA
3,57799351.158624927,2022-11-20,2022-11-20 06:05:28+00:00,first_visit,Почему ребёнок не понимает речь? Причины и способы коррекции,1,Трипвайер,1,Трипвайер1,TripA
4,81880761.161656716,2022-12-05,2022-12-05 17:30:08+00:00,first_visit,"Центр дистанционного и онлайн обучения и переподготовки логопедов ""Лого-Эксперт""",1,Главная,1,Главная1,MainA


21.2 Датафрейм с данными не покупателей<a id="step21_2"></a>

 [Вернуться в начало](#stepend)</n>

In [105]:
buyers_no_chain.head(5)

,ga_id,event_date,event_name,event_time,page_title,rang,utm_campaign,utm_medium,utm_source,type,rn,product_way,product_type
0,32725729.168023606,2023-06-05,page_view,2023-06-05 17:20:49+00:00,Логотека,39,reklama,podslysheno,vk,Курс,2,Курс2,KursB
1,32725729.168023606,2023-06-05,page_view,2023-06-05 17:21:52+00:00,Практические материалы для работы логопедов и коррекционных специалистов,40,reklama,podslysheno,vk,Курс,3,Курс3,KursC
2,40474410.167781444,2023-03-03,page_view,2023-03-03 04:31:48+00:00,25-12-2019 Учимся играя,8,(referral),referral,yandex.ru,Вебинар,2,Вебинар2,WebB
3,41947628.167842890,2023-03-10,page_view,2023-03-10 13:00:27+00:00,"Дракон, Паук и Осьминог – Логопедический игротренинг. Блог Лого-Эксперт",10,(direct),(none),(direct),Блог,9,Блог9,BlogI
4,41947628.167842890,2023-03-10,page_view,2023-03-10 13:06:06+00:00,Картотека дидактических игр. Блог Лого-Эксперт,11,(direct),(none),(direct),Блог,10,Блог10,BlogK


### `22. Цепи Маркова полный путь пользователей с объединением точек контакта после 10-ой`<a id="step22"></a>


22.1 Создадим функцию использующую цепи Маркова и на выходе получая матрицу вероятностей перехода и эффект удаления

In [106]:
def markov_chain_all(x,y):
# 1. Сформируем таблицу с путем до покупки
      chain_conv = x.copy()
# 1.2 Оставим только нужные колонки
      chain_conv = chain_conv[['ga_id', 'rang', 'type', 'product_type']]
# 1.3 Уберем строку с оплатой, это этап конверсии - conv (требуется для рассчета цепей)
      chain_conv = chain_conv.drop(chain_conv[chain_conv.type == "Оплата"].index)
# 1.4 Пустые значения заменим на empty
      chain_conv['product_type'] = chain_conv['product_type'].fillna('empty')
# 1.5 Сгруппируем каждый этап пути пользователя и преобразуем столбцец в список
      chain_conv = chain_conv.groupby('ga_id')['product_type'].agg(lambda x: x.tolist()).reset_index()
# 1.6 Переименуем столбец с путем пользователя для удобства
      chain_conv = chain_conv.rename(columns={"product_type": "path"})
# 1.7 Напишем функцию, вставляющую требуемый символ "">" между этапами пути пользователя
      def listToString(df):
          str1 = ""
          for i in df['path']:

              str1 += i + " > "
          return str1[:-3]
# 1.8 Применим функцию к столбцу с путем пользователя
      chain_conv['path'] = chain_conv.apply(listToString, axis=1)
# 1.9 Добавим в начало пути требуемый элемент
      chain_conv['path']=['start > ']+ chain_conv['path']
# 1.10 Добавим в конец пути требуемый элемент
      chain_conv['path']= chain_conv['path']+[' > conv']
      chain_conv = chain_conv.groupby('path').agg({'ga_id':'count'}).rename(columns={'ga_id': 'Conv'}).reset_index() # 1.11 Посчитаем кол-во каждого из получившихся путей пользователя
# 1.12 Оставим только требуемую колонку с путями пользователей до конверсии
      chain_conv = chain_conv[['path']]

# 2 Сформируем таблицу с путями пользователей, которые не совершили покупку
      chain_no_conv = y.copy()
# 2.2 Оставим только нужные колонки
      chain_no_conv = chain_no_conv[['ga_id', 'rang', 'product_way', 'product_type']]
 # 2.3 Сгруппируем каждый этап пути пользователя и преобразуем столбцец в список
      chain_no_conv = chain_no_conv.groupby('ga_id')['product_type'].agg(lambda x: x.tolist()).reset_index()
# 2.4 Переименуем столбец с путем пользователя для удобства
      chain_no_conv = chain_no_conv.rename(columns={"product_type": "path"})
# 2.5 Напишем функцию, вставляющую требуемый символ "">" между этапами пути пользователя
      def listToString_no(df):
          str1 = ""
          for i in df['path']:
              str1 += i + " > "
          return str1[:-3]
# 2.6 Применим функцию к столбцу с путем пользователя
      chain_no_conv['path'] = chain_no_conv.apply(listToString_no, axis=1)
# 2.7 Добавим в начало пути требуемый элемент
      chain_no_conv['path']=['start > ']+ chain_no_conv['path']
# 2.8 Добавим в конец пути требуемый элемент
      chain_no_conv['path']= chain_no_conv['path']+[' > null']
# 2.9 Посчитаем кол-во каждого из получившихся путей пользователя
      chain_no_conv = chain_no_conv.groupby('path').agg({'ga_id':'count'}).rename(columns={'ga_id': 'Null_Con'}).reset_index()
# 2.10 Оставим только требуемую колонку с путями пользователей до конверсии
      chain_no_conv = chain_no_conv[['path']]

# 3 Сформируем таблицу для построения цепей, добавим получившуюся таблицу к таблице из п.1
      markov_chain=pd.concat([chain_conv,chain_no_conv], axis=0, ignore_index=True)
# 3.1 Приведем данные в требуемый формат
      tab = pd.DataFrame(markov_chain)
 # 3.2 Применим к данным библиотеку для построения Цепей Маркова
      model = mma.run_model(paths=tab)
      rem_effect = model['removal_effects']
# 4 Эффект удаления каждой точки контакта, т.е если не расчитывать дополнительно, то это покажет важность каждой точки касания, и потерю конверсии если ее удалить
      rem = pd.DataFrame(rem_effect, index=['rem_effect'])
      rem = np.transpose(rem)
      rem = rem.sort_values('rem_effect', ascending=False)

# 5 Оценка вероятностей перехода к каждой точке контакта transition probabilities
      transition_probabilities = model['transition_matrix'].round(3)

      return transition_probabilities
      # return rem

In [107]:
markov_chain_all(buyers_chain, buyers_no_chain)

/usr/local/lib/python3.10/dist-packages/markov_model_attribution/__init__.py:121: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_df[col] = 0.00
/usr/local/lib/python3.10/dist-packages/markov_model_attribution/__init__.py:121: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_df[col] = 0.00
/usr/local/lib/python3.10/dist-packages/markov_model_attribution/__init__.py:121: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider 

,BlogG,WebG,KursL,ShowA,FunE,ShowC,LmI,WorkB,WebD,SubsF,BlogE,ShowB,TripF,WebK,FunF,SubsL,MkE,FunB,AkciaD,BlogF,SubsA,MkC,FunH,FunC,AkciaA,TripB,SubsH,IntA,FunL,MkK,TripL,KursC,FunA,BlogD,FunI,LmG,MkF,SuperB,IntK,WebC,ConfA,MkI,LmE,IntG,WorkA,MainA,TripI,AkciaC,LmF,WebA,MainB,MkG,WebF,MkB,TripD,LmD,KursG,KursE,IntF,IntD,KursK,TripH,WorkC,IntE,MkL,SubsK,SubsG,SubsC,MkH,BonusA,WebE,BlogK,BlogA,BlogL,MkD,IntI,WorkD,LmH,SubsI,KursH,BlogC,null,ShowD,TripE,WebH,IntL,KursI,SubsB,KursF,WebB,BlogH,FunG,KursA,SubsE,LmC,WebL,conv,BlogI,LmB,KursB,AkciaB,LmK,IntH,FunK,TripA,SuperA,FunD,SubsD,TripC,TripG,WebI,IntB,LmA,MkA,start,KursD,BlogB,IntC,empty,LmL,TripK
BlogG,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.009,0.001,0.000,0.000,0.003,0.001,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.026,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.017,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.842,0.000,0.073,0.000,0.000,0.002,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.013,0.000,0.000,0.000,0.000,0.000,0.000,0.003,0.001,0.002,0.0,0.000,0.000,0.001,0.0,0.000,0.000
WebG,0.000,0.000,0.009,0.000,0.000,0.000,0.000,0.000,0.000,0.004,0.000,0.000,0.000,0.000,0.000,0.000,0.004,0.000,0.000,0.000,0.000,0.004,0.000,0.000,0.000,0.004,0.000,0.000,0.000,0.004,0.000,0.009,0.000,0.000,0.000,0.000,0.013,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.004,0.000,0.000,0.000,0.000,0.000,0.000,0.013,0.000,0.000,0.004,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.013,0.000,0.000,0.004,0.000,0.000,0.000,0.000,0.000,0.000,0.053,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.004,0.580,0.009,0.000,0.058,0.000,0.000,0.000,0.000,0.000,0.004,0.000,0.000,0.022,0.000,0.004,0.027,0.004,0.000,0.000,0.000,0.000,0.000,0.000,0.004,0.035,0.000,0.000,0.004,0.009,0.049,0.0,0.000,0.000,0.040,0.0,0.000,0.000
KursL,0.000,0.007,0.671,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.020,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.034,0.000,0.007,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.007,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.094,0.000,0.000,0.000,0.000,0.000,0.007,0.000,0.000,0.000,0.007,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.054,0.000,0.007,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.013,0.007,0.007,0.000,0.013,0.007,0.000,0.000,0.000,0.000,0.000,0.000,0.020,0.000,0.000,0.000,0.013,0.000,0.0,0.000,0.000,0.000,0.0,0.007,0.000
ShowA,0.000,0.001,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.156,0.001,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.004,0.000,0.000,0.000,0.006,0.001,0.000,0.034,0.000,0.000,0.000,0.000,0.060,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.009,0.007,0.000,0.000,0.001,0.019,0.000,0.000,0.000,0.009,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.001,0.000,0.000,0.000,0.003,0.001,0.001,0.000,0.028,0.000,0.041,0.000,0.000,0.000,0.000,0.000,0.001,0.003,0.000,0.000,0.000,0.000,0.000,0.022,0.000,0.048,0.000,0.000,0.012,0.000,0.000,0.009,0.019,0.020,0.013,0.054,0.007,0.000,0.000,0.000,0.007,0.000,0.000,0.000,0.015,0.000,0.000,0.004,0.077,0.281,0.0,0.000,0.000,0.013,0.0,0.000,0.000
FunE,0.000,0.000,0.000,0.017,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.783,0.000,0.009,0.000,0.000,0.000,0.009,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.009,0.000,0.000,0.000,0.000,0.009,0.000,0.000,0.000,0

>**Комментарий**: *Из полной матрицы вероятностей сложно что-то понять и сделать выводы а урезание точек контакта приведет к зацикленности путей пользователя. Следующим шагом в анализе может быть выделение пользователей по типу покупки, но наиболее полезным остается это removable effect, который дает представление о важности каждой точки контакта*

>**Комментарий**: *Removable Effect аналогично ранжирует точки контакта, как и в короткой версии цепей в пункте ниже*

### `23. Цепи Маркова короткие с только уникальными типами точек контакта, без нумерации`<a id="step23"></a>

 [Вернуться в начало](#stepend)</n>

23.1 Создадим функцию использующую цепи Маркова и на выходе получая матрицу вероятностей перехода и эффект удаления


 [Вернуться в начало](#stepend)</n>

In [108]:
def markov_chain_short(x,y):

# 1 Сформируем таблицу с пользователями, которые совершили покупки
  buyers_chain_short = x.copy()
  buyers_chain_short['rn'] = buyers_chain_short['rn'].astype(str)
  buyers_chain_short['rang'] = buyers_chain_short['rang'].astype(str)
  buyers_chain_short = buyers_chain_short.query('rn == "1"').reset_index()
  buyers_chain_short = buyers_chain_short[['ga_id', 'event_date', 'event_time', 'rang', 'page_title', 'type']]

# 2 Сформируем таблицу с пользователями, которые не совершили покупки
  buyers_no_chain_short = y.copy()
  buyers_no_chain_short['rn'] = buyers_no_chain_short['rn'].astype(str)
  buyers_no_chain_short['rang'] = buyers_no_chain_short['rang'].astype(str)
  buyers_no_chain_short = buyers_no_chain_short.query('rn == "1"').reset_index()
  buyers_no_chain_short = buyers_no_chain_short[['ga_id', 'event_date', 'event_time', 'rang', 'page_title', 'type']]
  buyers_no_chain_short = buyers_no_chain_short.sort_values(["ga_id"])

# 3. Сформируем таблицу с путем до покупки
  chain_conv = buyers_chain_short.copy()
# 3.1 Оставим только нужные колонки
  chain_conv = chain_conv[['ga_id', 'rang', 'type']]
# 3.2 Заменим названия
  chain_conv['type'] = chain_conv['type'].replace(['Акция', 'Блог', 'Бонусы', 'Вебинар', 'Главная', 'Интенсив', 'Конференция', 'Курс',
                                          'Лид-магнит', 'Логошоу', 'Мастер-класс', 'Нескучная логопедия', 'Подписка', 'Рабочая программа',
                                          'Супервизия', 'Трипвайер', 'Оплата', 'Вебинар_апселл', 'Вебинары', 'Заявки на вебинар'],
                                            ['Akcia', 'Blog', 'Bonus', 'Web', 'Main', 'Intensiv', 'Conf', 'Kurs', 'LM', 'Show', 'Mk', 'Fun',
                                            'Subs', 'Wordprog', 'Supervizia', 'Trip', 'Oplata', 'Web', 'Web', 'Web' ])
# 3.3 Уберем строку с оплатой, это этап конверсии - conv (требуется для рассчета цепей)
  chain_conv = chain_conv.drop(chain_conv[chain_conv.type == "Oplata"].index)
# 3.4 Сгруппируем каждый этап пути пользователя и преобразуем столбцец в список
  chain_conv = chain_conv.groupby('ga_id')['type'].agg(lambda x: x.tolist()).reset_index()
# 3.5 Переименуем столбец с путем пользователя для удобства
  chain_conv = chain_conv.rename(columns={"type": "path"})
# 3.6 Напишем функцию, вставляющую требуемый символ "">" между этапами пути пользователя
  def listToString(df):
          str1 = ""
          for i in df['path']:
            str1 += i + " > "
          return str1[:-3]
# 3.7 Применим функцию к столбцу с путем пользователя
  chain_conv['path'] = chain_conv.apply(listToString, axis=1)
 # 3.8 Добавим в начало пути требуемый элемент
  chain_conv['path']=['start > ']+ chain_conv['path']
# 3.9 Добавим в конец пути требуемый элемент
  chain_conv['path']= chain_conv['path']+[' > conv']
# 3.10 Посчитаем кол-во каждого из получившихся путей пользователя
  chain_conv = chain_conv.groupby('path').agg({'ga_id':'count'}).rename(columns={'ga_id': 'Conv'}).reset_index()
# 3.11 Оставим только требуемую колонку с путями пользователей до конверсии
  chain_conv = chain_conv[['path']]

# 4 Сформируем таблицу с путями пользователей, которые не совершили покупку
  chain_no_conv = buyers_no_chain_short.copy()
# 4.1 Оставим только нужные колонки
  chain_no_conv = chain_no_conv[['ga_id', 'rang', 'type']]
# 4.2 Заменим названия
  chain_no_conv['type'] = chain_no_conv['type'].replace(['Акция', 'Блог', 'Бонусы', 'Вебинар', 'Главная', 'Интенсив', 'Конференция', 'Курс',
                                          'Лид-магнит', 'Логошоу', 'Мастер-класс', 'Нескучная логопедия', 'Подписка', 'Рабочая программа',
                                          'Супервизия', 'Трипвайер', 'Оплата', 'Вебинар_апселл', 'Вебинары', 'Заявки на вебинар'],
                                            ['Akcia', 'Blog', 'Bonus', 'Web', 'Main', 'Intensiv', 'Conf', 'Kurs', 'LM', 'Show', 'Mk', 'Fun',
                                            'Subs', 'Wordprog', 'Supervizia', 'Trip', 'Oplata', 'Web', 'Web', 'Web' ])
# 4.3 Сгруппируем каждый этап пути пользователя и преобразуем столбцец в список
  chain_no_conv = chain_no_conv.groupby('ga_id')['type'].agg(lambda x: x.tolist()).reset_index()
# 4.4 Переименуем столбец с путем пользователя для удобства
  chain_no_conv = chain_no_conv.rename(columns={"type": "path"})
# 4.5 Напишем функцию, вставляющую требуемый символ "">" между этапами пути пользователя
  def listToString_no(df):
          str1 = ""
          for i in df['path']:
              str1 += i + " > "
          return str1[:-3]
# 4.6 Применим функцию к столбцу с путем пользователя
  chain_no_conv['path'] = chain_no_conv.apply(listToString_no, axis=1)
# 4.7 Добавим в начало пути требуемый элемент
  chain_no_conv['path']=['start > ']+ chain_no_conv['path']
# 4.8 Добавим в конец пути требуемый элемент
  chain_no_conv['path']= chain_no_conv['path']+[' > null']
# 4.9 Посчитаем кол-во каждого из получившихся путей пользователя
  chain_no_conv = chain_no_conv.groupby('path').agg({'ga_id':'count'}).rename(columns={'ga_id': 'Null_Con'}).reset_index()
# 4.10 Оставим только требуемую колонку с путями пользователей до конверсии
  chain_no_conv = chain_no_conv[['path']]

# 5 Сформируем таблицу для построения цепей, добавим получившуюся таблицу в п.4 к таблице из п.3
  markov_chain=pd.concat([chain_conv,chain_no_conv], axis=0, ignore_index=True)
# 5.1 Приведем данные в требуемый формат
  tab = pd.DataFrame(markov_chain)
# 5.2 Применим к данным библиотеку для построения Цепей Маркова
  model = mma.run_model(paths=tab)
  rem_effect = model['removal_effects']

# 6 Эффект удаления каждой точки контакта, т.е если не расчитывать дополнительно, то это покажет важность каждой точки касания, и потерю конверсии если ее удалить
  rem = pd.DataFrame(rem_effect, index=['rem_effect'])
  rem = np.transpose(rem)
  rem = rem.sort_values('rem_effect', ascending=False)

# 7 Оценка вероятностей перехода к каждой точке контакта transition probabilities
#    transition_probabilities = model['transition_matrix'].round(3)

  # return transition_probabilities
  return  rem

In [109]:
markov_chain_short(buyers_chain, buyers_no_chain)

,rem_effect
Mk,0.480702
Intensiv,0.465880
Web,0.450232
Trip,0.446667
Kurs,0.426427
Main,0.391710
Subs,0.335787
Blog,0.314281
LM,0.289987
Akcia,0.255227


>**Комментарий**: *Блог яв-ся первой страницей после страниц, где возможно совершить покупку. Есть основания говорить о том, данные цепей Маркова подтверждают выводы из п.13*


### `24. Выводы и дальнейшие шаги`<a id="step24"></a>

 [Вернуться в начало](#stepend)</n>

**Выводы**:
1. В среднем пользователю требуется 52 дня, чтобы совершить первую покупки

2. Кол-во дней до первой покупки постепенно снижается, от когорты к когорте

3. Сравнивая посещения страниц по типам продукта, ключевая разница между теми, кто купил и теми, кто не купил в кол-ве посещений страниц блога. 6 страниц у купивших и 3 у не купивших

4. Кол-во посещенных страниц в зависимости от первой покупки

* Трипвайер - 3 Лид-магнита, 2 Блога, 2 Вебинара, 2 Интенсива, 2 Мастер-класса, 1 Подписка
* Вебинар - 1 Лид-магнит, 4 Блога, 3 Вебинара, 2 Интенсива, 3 Мастер-класса, 1 Подписка
* Практикум - 1 Лид-магнит, 2 Блога, 2 Вебинара, 4 Интенсива, 4 Мастер-класса, 3 Подписка
* Интенсив - 2 Лид-магнит, 2 Блога, 2 Вебинара, 1 Интенсив, 2 Мастер-класса, 1 Подписка
* Курс - 2 Лид-магнит, 8 Блога, 2 Вебинара, 2 Интенсив, 2 Мастер-класса, 1 Подписка

Ниже, границы с периодами до покупки по типу продукта

* Трипвайер - Среднее кол-во дней до покупки 0-44
* Вебинар - Среднее кол-во дней до покупки 1-22
* Практикум - Среднее кол-во дней до покупки 5-179
* Интенсив - Среднее кол-во дней до покупки 1-19
* Курс - Среднее кол-во дней до покупки 2-18

5. Цепи Маркова подтверждают важность страницы блога перед покупкой, нужно проработать ее

6. Из данных видно, что акции сокращают кол-во дней до покупки. Явно не наблюдается зависимость кол-ва дней до покупки от стоимости продукта. У большинства пользователей фигурируют Блог и посещение практически всех страниц продуктов. Т.е перед совершением покупки пользователь изучает весь спектр продуктов.

7. Наблюдается, что перед покупкой Курса, пользователь посещает самое большое кол-во страниц блога. Связующие страницы — это лид-магнит и блог, скорее всего они яв-ся стартовыми для знакомства. Плюс, пользователь перед покупкой посещает все категории продуктов портала.

**Дальнейшие шаги**:

1. Важно провести исследование того, на какую работу нанимают каждый тип продукта. Из данных видно, хотя только из двух покупателей, у того кто купил Вебинар, сначала было изучение блога, затем страницы продуктов. А у того, кто купил Курс - изучение блога было перед покупкой. Скорее всего, тот кто купил Вебинар решал какую-то одну узкую задачу и пользователь искал ее решение, при этом искал решение бесплатно.

2. Нужно провести качественные исследования купивших и не купивших, какую работу выполняет каждая страница, чего не хватает/хватает. Изучить поведение на странице блога и сгенерировать гипотезы переработки страницы блога, — добавив элементы перевода на смежные страницы, продукты, с разделением по направлениям и экспертам

3. Реализовать возможность более легкого переключения на другие страницы с каждой страницы, создав перекрестные линки, поп-апы и онлайн-чаты (типа Carrot Quest)

Перейти в другой анализ [Анализ поведения пользователей после первой покупки](https://nbviewer.org/github/artem-ilienkov/product_analysis_ed_tech/blob/main/ed_tech_sales.ipynb)

### [Вернуться в начало](#stepend)</n>